### Trendyol Kargo Desi

In [1]:
# pip install pandas

In [2]:
import unicodedata
import datetime as dt
import os
import sys
import re
import numpy as np
import pandas as pd
import math
from pathlib import Path
import nltk
from nltk import word_tokenize 
import shutil
from os.path import isfile, join
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import locale
from locale import atof
from geopy.geocoders import Nominatim
from geopy.point import Point
from geopy.exc import GeocoderTimedOut
from functools import reduce

In [3]:
os.getcwd()

'c:\\Users\\user\\Desktop\\Data Analysis\\Data-Scientist\\Work\\Adoba\\Sales_Report\\ReDe'

#### Functions

In [4]:
# custom character alphabet for word
tr = re.compile(r"[abcçdefgğhıijklmnoöprsştuüvyzqxw]+", re.IGNORECASE|re.UNICODE) # Turkish filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n0123456789'
en = re.compile(r"[abcdefghıijklmnopqrstxuvwyz]+", re.IGNORECASE|re.UNICODE) # English
nl = re.compile(r"[abcdefghıijklmnopqrstxuvwyzāăēĕīĭōŏūŭ]+", re.IGNORECASE|re.UNICODE) # Dutch (Flemenk)
fr = re.compile(r"[abcçdefghıijklmnopqrstxuvwyzàâæèéêëîïôœùûüÿ]+", re.IGNORECASE|re.UNICODE) # French
de = re.compile(r"[abcdefghıijklmnopqrstxuvwyzäöüß]+", re.IGNORECASE|re.UNICODE) # German
es = re.compile(r"[abcdefghıijklmnopqrstxuvwyzñáéíóú]+", re.IGNORECASE|re.UNICODE) # Spanish (¿¡)
pt = re.compile(r"[abcçdefghıijklmnopqrstxuvwyzàáâãéêíóôõú]+", re.IGNORECASE|re.UNICODE) # Portuguese
it = re.compile(r"[abcdefghıijklmnopqrstxuvwyzàéèìòùî]+", re.IGNORECASE|re.UNICODE) # Italian
ar = re.compile(r"[ٿصؼۤڳڲؿڎػڠجڿ٬ٸؽؒؓطۄڀۂؘؔتٚڛےٝڜؖڦ٫ډ۰زۇٖۀ،لۓعٮێڔ۶ؚۧۜڤۏإٞٷؗۖ؈ژۣؕؑٴأۻڸۺگاڴڹۯ؉ْڌ؍ي؟ـٟړۅؐڶُىڽېًۢؠضۚڄٛڏٱۦ٩س٦ڼڂٔۘ٠ښٌٍ۬ٳ۾ٲږذۋٵٜ٘ڞڅںٗهڣۿپڒۥۗڋیؙم؞ثۨٹڵڪظٶۭ١ڭەڨحٕ؎ٺڷٰ۪۫ڻڥۛڑڟټآڡغګ؊ّٯڧڮ؏ۮ؋ؤ٪ؾڗۼق۟دکوِڰڐۃ۽ہفرڇچڝ۴بۈٽڕۡھةٓڃئ؛ڬٙڙڢڱۊَۆۉځ۠ۍۑۙڊنءڈٻشڍ؇۵كخ\ا]+", re.IGNORECASE|re.UNICODE) # Arabic

In [5]:
def convert_one_character_letter(text):
    '''This function converts two byte occupy of letter to one byte unicode character without any visual change \n
    like as Turkish character ç,ş,ö,ğ. \n
    convert_one_character_letter(text): text is any string word or sentence.
    '''
    new_text = unicodedata.normalize('NFC', f"{text}")
    return new_text

In [6]:
def lower_func(text):
    '''This function convert string character to lowercase. \n
    lower_func(text=str): text is any string word or sentence
    '''
    string_lower = str.lower(text)
    return string_lower

In [7]:
def whitespace_del(text):
    '''This function provides removing left right white space.\n
    whitespace_del(text): text is a string value as word or sentence.
    '''
    text_var = str(text)
    text_var = text_var.strip()  # rstrip lstrip
    return text_var

In [8]:
# PTT, TEX için 0 TL - 124,99 TL arası siparişlerinizde 19,58 TL + KDV;
# 
# 125 TL - 199,99 TL arası siparişlerinizde 37,49 TL + KDV'dir.
# 
# * Mng, Aras, Sendeo, Sürat için 0 TL - 124,99 TL arası siparişlerinizde 25,83 TL + KDV ;
# 
# 125 TL - 199,99 TL arası siparişlerinizde 44,99 TL + KDV'dir.
# 
# * Yurtiçi Kargo için; 0 TL - 124,99 TL arası siparişlerinizde 43,33 TL + KDV ;
# 
# 125 TL - 199,99 TL arası siparişlerinizde 59,58 TL + KDV'dir.

In [9]:
def cargo_price_check(cargo_brand, sales_price):
    '''This function is check cargo brand and product sales price for cargo price \n. 
    cargo_price_check(cargo_brand, sales_price)'''
    
    result_price = np.nan

    if sales_price >= 0 and sales_price <= 199.99:
        if ((cargo_brand == 'PTT') or (cargo_brand == 'TEX')) and (sales_price >= 0 and sales_price <= 124.99):
                result_price = 19.58
        elif ((cargo_brand == 'PTT') or (cargo_brand == 'TEX')) and (sales_price >= 125 and sales_price <= 199.99):
                result_price = 37.49

        if ((cargo_brand == 'MNG') or (cargo_brand == 'Aras') or (cargo_brand == 'Sendeo') or (cargo_brand =='Surat')) and (sales_price >= 0 and sales_price <= 124.99):
            result_price = 25.83
        elif ((cargo_brand == 'MNG') or (cargo_brand == 'Aras') or (cargo_brand == 'Sendeo') or (cargo_brand =='Surat')) and (sales_price >= 125 and sales_price <= 199.99):
            result_price = 44.99

        if (cargo_brand == 'Yurtiçi') and (sales_price >= 0 and sales_price <= 124.99):
            result_price = 43.33
        elif (cargo_brand == 'Yurtiçi') and (sales_price >= 125 and sales_price <= 199.99):
            result_price = 59.58
    else:
        result_price = np.nan
    
    return result_price

#### Read Data

In [124]:
df_kargo = pd.read_excel(r"C:\Users\user\Desktop\Data Analysis\Adoba\Sales_Report\Data\Trendyol\Kargo\30.07.2024\trendyol kargo.xlsx")
df_kargo.sample(5)

,Barkod,Paket No,Kargo Firması,Sipariş Numarası,İl,İlçe,Ürün Adı,Komisyon Oranı,Marka,Adet,Birim Fiyatı,Satış Tutarı,İndirim Tutarı,Trendyol İndirim Tutarı,Faturalanacak Tutar,Kargodan alınan desi,Faturalanan Kargo Tutarı
5858,8680998035898,1960486511,Trendyol Express Marketplace,2439694769,İstanbul,Ümraniye,Teaplus Kırmızı Elektrikli Cam Demlikli Çaycı ...,14.00,AWOX,1,448.00,448.00,0.0,0.0,448.00,8.0,94.06
13817,EV1819190520248,1998041435,Trendyol Express Marketplace,9033526417,İstanbul,Güngören,"Ev18 Doğal Ağaç Çocuk Oyun Evi Demonte, Hediye...",13.22,ROZİTOYS,1,1975.00,1975.00,0.0,0.0,1975.00,9.0,97.62
75520,8680067147850,2249774243,Trendyol Express Marketplace,9318250021,Antalya,Manavgat,"Küçük Boy Kedi Tırnak Makası 8680067147850, on...",18.31,MİO,1,37.63,37.63,0.0,0.0,37.63,NaN,23.49
78045,254781485795,2260580645,Trendyol Express Marketplace,9330434052,Bursa,Osmangazi,"Ahşap Özel Isim Baskılı, Kedi Mama Seramik Su ...",18.31,GOBYPET,1,250.00,250.00,0.0,0.0,250.00,3.0,Henüz fatura kesilmemiştir.
33200,8680998011022,2104022102,Trendyol Express Marketplace,9152360493,Kahramanmaraş,Elbistan,Dual Elektrikli Çay Makinesi Beyaz-kırmızı DUA...,3.20,AWOX,1,278.00,278.00,0.0,0.0,278.00,7.0,88.35


In [125]:
df_kargo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88796 entries, 0 to 88795
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Barkod                    88796 non-null  object 
 1   Paket No                  88796 non-null  int64  
 2   Kargo Firması             88795 non-null  object 
 3   Sipariş Numarası          88796 non-null  int64  
 4   İl                        88796 non-null  object 
 5   İlçe                      88796 non-null  object 
 6   Ürün Adı                  88796 non-null  object 
 7   Komisyon Oranı            88792 non-null  float64
 8   Marka                     88678 non-null  object 
 9   Adet                      88796 non-null  int64  
 10  Birim Fiyatı              88796 non-null  float64
 11  Satış Tutarı              88796 non-null  float64
 12  İndirim Tutarı            88796 non-null  float64
 13  Trendyol İndirim Tutarı   88796 non-null  float64
 14  Fatura

In [126]:
df_kargo.columns

Index(['Barkod', 'Paket No', 'Kargo Firması', 'Sipariş Numarası', 'İl', 'İlçe',
       'Ürün Adı', 'Komisyon Oranı', 'Marka', 'Adet', 'Birim Fiyatı',
       'Satış Tutarı', 'İndirim Tutarı', 'Trendyol İndirim Tutarı',
       'Faturalanacak Tutar', 'Kargodan alınan desi',
       'Faturalanan Kargo Tutarı'],
      dtype='object')

In [127]:
df_kargo2 = df_kargo.drop_duplicates(subset="Paket No")
df_kargo2 = df_kargo2.reset_index(drop=True)
df_kargo2.head(5)

,Barkod,Paket No,Kargo Firması,Sipariş Numarası,İl,İlçe,Ürün Adı,Komisyon Oranı,Marka,Adet,Birim Fiyatı,Satış Tutarı,İndirim Tutarı,Trendyol İndirim Tutarı,Faturalanacak Tutar,Kargodan alınan desi,Faturalanan Kargo Tutarı
0,97815659247966334,1935755636,Trendyol Express Marketplace,8061482665,Aydın,Germencik,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, Sakl...",20.34,ALİTECH,1,375.0,375.0,0.0,0.0,375.0,9.0,97.62
1,ALS3463102806,1935764998,Trendyol Express Marketplace,2424519644,Adıyaman,Gölbaşı,3'lü Çok Amaçlı Kutu Mega Çizgili Kahverengi 6...,16.20,ALAS,1,350.0,350.0,0.0,0.0,350.0,10.0,103.37
2,604808214197,1935781088,MNG Kargo Marketplace,8061494292,İstanbul,Çekmeköy,"Ahşap Özel İsim Baskılı, Kare Kedi Köpek Mama ...",18.31,GOBYPET,1,149.0,149.0,0.0,0.0,149.0,2.0,35.49
3,ÇG742305202381,1935783700,MNG Kargo Marketplace,2424528751,İstanbul,Kadıköy,"Çg74 Ahşap Çocuk Makyaj Masası Tabureli,unicor...",16.27,ROZİTOYS,1,1200.0,1200.0,0.0,0.0,1200.0,10.0,112.79
4,868407600769,1935789958,MNG Kargo Marketplace,8061497847,İstanbul,Ümraniye,Çömlek Ayaklı Sırlı Seramik Kedi Maması Kabı 2...,18.31,GOBYPET,1,160.0,160.0,0.0,0.0,160.0,4.0,71.99


In [128]:
df_kargo2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80169 entries, 0 to 80168
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Barkod                    80169 non-null  object 
 1   Paket No                  80169 non-null  int64  
 2   Kargo Firması             80168 non-null  object 
 3   Sipariş Numarası          80169 non-null  int64  
 4   İl                        80169 non-null  object 
 5   İlçe                      80169 non-null  object 
 6   Ürün Adı                  80169 non-null  object 
 7   Komisyon Oranı            80166 non-null  float64
 8   Marka                     80062 non-null  object 
 9   Adet                      80169 non-null  int64  
 10  Birim Fiyatı              80169 non-null  float64
 11  Satış Tutarı              80169 non-null  float64
 12  İndirim Tutarı            80169 non-null  float64
 13  Trendyol İndirim Tutarı   80169 non-null  float64
 14  Fatura

In [129]:
df_kargo2 = df_kargo2.dropna(subset="Kargodan alınan desi")
df_kargo2.reset_index(drop=True, inplace=True)
df_kargo2.head(5)

,Barkod,Paket No,Kargo Firması,Sipariş Numarası,İl,İlçe,Ürün Adı,Komisyon Oranı,Marka,Adet,Birim Fiyatı,Satış Tutarı,İndirim Tutarı,Trendyol İndirim Tutarı,Faturalanacak Tutar,Kargodan alınan desi,Faturalanan Kargo Tutarı
0,97815659247966334,1935755636,Trendyol Express Marketplace,8061482665,Aydın,Germencik,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, Sakl...",20.34,ALİTECH,1,375.0,375.0,0.0,0.0,375.0,9.0,97.62
1,ALS3463102806,1935764998,Trendyol Express Marketplace,2424519644,Adıyaman,Gölbaşı,3'lü Çok Amaçlı Kutu Mega Çizgili Kahverengi 6...,16.20,ALAS,1,350.0,350.0,0.0,0.0,350.0,10.0,103.37
2,604808214197,1935781088,MNG Kargo Marketplace,8061494292,İstanbul,Çekmeköy,"Ahşap Özel İsim Baskılı, Kare Kedi Köpek Mama ...",18.31,GOBYPET,1,149.0,149.0,0.0,0.0,149.0,2.0,35.49
3,ÇG742305202381,1935783700,MNG Kargo Marketplace,2424528751,İstanbul,Kadıköy,"Çg74 Ahşap Çocuk Makyaj Masası Tabureli,unicor...",16.27,ROZİTOYS,1,1200.0,1200.0,0.0,0.0,1200.0,10.0,112.79
4,868407600769,1935789958,MNG Kargo Marketplace,8061497847,İstanbul,Ümraniye,Çömlek Ayaklı Sırlı Seramik Kedi Maması Kabı 2...,18.31,GOBYPET,1,160.0,160.0,0.0,0.0,160.0,4.0,71.99


#### Duplicated

In [130]:
df_prod_barkod = df_kargo2[["Barkod","Ürün Adı"]]
df_prod_barkod = df_prod_barkod.drop_duplicates()
df_prod_barkod = df_prod_barkod.reset_index(drop=True)
df_prod_barkod

,Barkod,Ürün Adı
0,97815659247966334,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, Sakl..."
1,ALS3463102806,3'lü Çok Amaçlı Kutu Mega Çizgili Kahverengi 6...
2,604808214197,"Ahşap Özel İsim Baskılı, Kare Kedi Köpek Mama ..."
3,ÇG742305202381,"Çg74 Ahşap Çocuk Makyaj Masası Tabureli,unicor..."
4,868407600769,Çömlek Ayaklı Sırlı Seramik Kedi Maması Kabı 2...
...,...,...
1634,868407601702,Makaralı Otomatik 5 Metre Geri Çekilebilen Ked...
1635,TYBE094QYA434T8008,Orijinal Sayaçlı El Yayı 60 Kg - 5 Kg Ayarlana...
1636,8680998014153,Turkuaz Kafija Kahve Makinesi New Sesli Ve Işı...
1637,0043561372346,6'lı Yün Kurutma Topu Doğal Çamaşır Yumuşatıcı...


In [131]:
df_prod_barkod["Ürün Adı"].nunique()

1424

In [132]:
df_barkod_sort = df_prod_barkod.sort_values(by="Barkod")
df_barkod_sort = df_barkod_sort.reset_index(drop=True)
df_barkod_sort

,Barkod,Ürün Adı
0,000416648254,"Pembe Ahşap Kaydıraklı Oyun Evi, Ahşap Oyun Ev..."
1,000437943412,Çamaşır Kurutma Makinası İçin Uçucu Yağlar Set...
2,0006941057446691,Hayvanlı Kumsuz Hacıyatmaz Kaplan 98 Cm X 44 C...
3,002154600346,"Dekoratif Salyangoz Jüt Dokuma Bardak Altlığı,..."
4,0021547145646,Su Sıvı Geçirmez Yıkanabilir Hasta Yaşlı Bakım...
...,...,...
1634,ÇG842302202341,"Çg84 Yeşil Ahşap Ledli Çocuk Mutfağı Pembe, Ah..."
1635,ÇG912305202332,"Çg91 Pembe Ahşap Ledli Çocuk Mutfağı Pembe, Ah..."
1636,ÇG912305202332,"Çg91 Pembe Ahşap Ledli Çocuk Mutfağı Pembe, Ah..."
1637,ÇakmakGazı11042023-4,Çakmak Gazı 270 Ml. Parlı Kırmızı Kutu 3'lü Se...


In [133]:
# df_barkod_sort.to_excel(r"C:\Users\user\Downloads\Trendyol Kargo Ürün Barkod.xlsx", index=False)

In [134]:
df_barkod_sort_dup =  df_barkod_sort[df_barkod_sort["Barkod"].duplicated(keep=False)]
# df_barkod_sort_dup = df_barkod_sort[df_barkod_sort["Barkod"].duplicated()]
df_barkod_sort_dup

,Barkod,Ürün Adı
31,003084gri8400,"Pencereli Kazak Hurcu 45x40x30 Gri 003084, one..."
32,003084gri8400,Pencereli Kazak Hurcu Gri 45x40x30 - Kahvereng...
69,00341894644531,Modern Line 3 Parça Battaniye Seti - Bej 00341...
70,00341894644531,Modern Line 3 Parça Battaniye Seti - Bej 86825...
109,003562479456,"Sole 750 W Mini Tost Makinesi, Seramik Kaplama..."
...,...,...
1632,ÇG832305202312,"Çg83 Pembe Ahşap Çocuk Mutfağı Pembe, Ahşap Mu..."
1633,ÇG842302202341,"Çg84 Yeşil Ahşap Ledli Çocuk Mutfağı Pembe, Ah..."
1634,ÇG842302202341,"Çg84 Yeşil Ahşap Ledli Çocuk Mutfağı Pembe, Ah..."
1635,ÇG912305202332,"Çg91 Pembe Ahşap Ledli Çocuk Mutfağı Pembe, Ah..."


In [135]:
# df_barkod_sort_dup.to_excel(r"C:\Users\user\Downloads\Trendyol Kargo Barkod Tekrar.xlsx", index=False)

In [136]:
df_prod_sort = df_prod_barkod.sort_values(by="Ürün Adı")
df_prod_sort = df_prod_sort.reset_index(drop=True)
df_prod_sort

,Barkod,Ürün Adı
0,yüz487514280114,"%100 Doğal Keçi Kılı Yüz Bakım Fırçası, Yüz Ba..."
1,8684076009987,%100 Doğal Sarı Kantaron Yağı 50 ml TYCCV4VKIN...
2,8684076000083,"%100 Doğal Soğuk Sıkım Sabit Çörekotu Yağı, Bl..."
3,8684076000076,"%100 Doğal Çam Terebentin Yağı, Pine Turpentin..."
4,2997778555322,0-3 Yaş Zeka Kartları Dokun Hisset Tanı Eğitim...
...,...,...
1634,985524140,Şişme Tabanlı 3 Boğumlu Sunset Havuz 114x25 cm...
1635,TYBHIT32Q6PE8REX10,Şişme Tabanlı 3 Boğumlu Sunset Havuz 114x25 cm...
1636,EDOT-0572-0016,Şönil Kilim Desenli Modern Halı (MAKİNADA YIKA...
1637,EDOT-0572-0030,Şönil Kilim Desenli Modern Halı (MAKİNADA YIKA...


In [137]:
df_prod_sort_dup = df_prod_sort[df_prod_sort["Ürün Adı"].duplicated(keep=False)]
# df_prod_sort_dup = df_prod_sort[df_prod_sort["Ürün Adı"].duplicated()]
df_prod_sort_dup

,Barkod,Ürün Adı
66,8699243467620-LY1,3 Gözlü-bölmeli Çekmece Yeni Sezon Nano Teknol...
67,8699243467620-LY3,3 Gözlü-bölmeli Çekmece Yeni Sezon Nano Teknol...
68,8699243467620-LY1,3 Gözlü-bölmeli Çekmece Yeni Sezon Nano Teknol...
69,8699243467620-LY3,3 Gözlü-bölmeli Çekmece Yeni Sezon Nano Teknol...
77,3-8699243469945-LY1,3 Lü Çok Amaçlı Baza Kutusu Seti Yeni Sezon Su...
...,...,...
1593,8434546721,Çömlek Ayaklı Sırlı Seramik Kedi Maması Kabı 0...
1616,11442012345647,"İsme Özel, Yıkanabilir Kedi, Köpek Çadırı, Kol..."
1617,798421352688941,"İsme Özel, Yıkanabilir Kedi, Köpek Çadırı, Kol..."
1618,11442012345647,"İsme Özel, Yıkanabilir Kedi, Köpek Çadırı, Kol..."


In [138]:
# df_prod_sort_dup.to_excel(r"C:\Users\user\Downloads\Trendyol Kargo Ürün Tekrar.xlsx", index=False)

#### 1 Adet 

In [139]:
df_kargo3 = df_kargo2[df_kargo2["Adet"] == 1]
df_kargo3 = df_kargo3.reset_index(drop=True)
df_kargo3.head(5)

,Barkod,Paket No,Kargo Firması,Sipariş Numarası,İl,İlçe,Ürün Adı,Komisyon Oranı,Marka,Adet,Birim Fiyatı,Satış Tutarı,İndirim Tutarı,Trendyol İndirim Tutarı,Faturalanacak Tutar,Kargodan alınan desi,Faturalanan Kargo Tutarı
0,97815659247966334,1935755636,Trendyol Express Marketplace,8061482665,Aydın,Germencik,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, Sakl...",20.34,ALİTECH,1,375.0,375.0,0.0,0.0,375.0,9.0,97.62
1,ALS3463102806,1935764998,Trendyol Express Marketplace,2424519644,Adıyaman,Gölbaşı,3'lü Çok Amaçlı Kutu Mega Çizgili Kahverengi 6...,16.20,ALAS,1,350.0,350.0,0.0,0.0,350.0,10.0,103.37
2,604808214197,1935781088,MNG Kargo Marketplace,8061494292,İstanbul,Çekmeköy,"Ahşap Özel İsim Baskılı, Kare Kedi Köpek Mama ...",18.31,GOBYPET,1,149.0,149.0,0.0,0.0,149.0,2.0,35.49
3,ÇG742305202381,1935783700,MNG Kargo Marketplace,2424528751,İstanbul,Kadıköy,"Çg74 Ahşap Çocuk Makyaj Masası Tabureli,unicor...",16.27,ROZİTOYS,1,1200.0,1200.0,0.0,0.0,1200.0,10.0,112.79
4,868407600769,1935789958,MNG Kargo Marketplace,8061497847,İstanbul,Ümraniye,Çömlek Ayaklı Sırlı Seramik Kedi Maması Kabı 2...,18.31,GOBYPET,1,160.0,160.0,0.0,0.0,160.0,4.0,71.99


In [140]:
df_kargo3["Ürün Adı"].nunique()

1388

In [141]:
df_kargo3["Barkod"].nunique()

1384

In [142]:
df_kargo3.columns

Index(['Barkod', 'Paket No', 'Kargo Firması', 'Sipariş Numarası', 'İl', 'İlçe',
       'Ürün Adı', 'Komisyon Oranı', 'Marka', 'Adet', 'Birim Fiyatı',
       'Satış Tutarı', 'İndirim Tutarı', 'Trendyol İndirim Tutarı',
       'Faturalanacak Tutar', 'Kargodan alınan desi',
       'Faturalanan Kargo Tutarı'],
      dtype='object')

#### Groupby Min Max Mean Most Freq

###### Value Count

In [143]:
df_kargo_val_count = df_kargo3.groupby("Ürün Adı")[["Kargodan alınan desi"]].value_counts().reset_index()
df_kargo_val_count

,Ürün Adı,Kargodan alınan desi,count
0,"%100 Doğal Keçi Kılı Yüz Bakım Fırçası, Yüz Ba...",1.0,2
1,"%100 Doğal Keçi Kılı Yüz Bakım Fırçası, Yüz Ba...",2.0,2
2,"%100 Doğal Keçi Kılı Yüz Bakım Fırçası, Yüz Ba...",5.0,1
3,%100 Doğal Sarı Kantaron Yağı 50 ml TYCCV4VKIN...,2.0,2
4,%100 Doğal Sarı Kantaron Yağı 50 ml TYCCV4VKIN...,9.0,1
...,...,...,...
5203,Şönil Kilim Desenli Modern Halı (MAKİNADA YIKA...,10.0,3
5204,Şönil Kilim Desenli Modern Halı (MAKİNADA YIKA...,4.0,4
5205,Şönil Kilim Desenli Modern Halı (MAKİNADA YIKA...,5.0,3
5206,Şönil Kilim Desenli Modern Halı (MAKİNADA YIKA...,3.0,1


In [144]:
# df_kargo_val_count["Ürün Adı"].fillna(method='ffill')  # fill nan value with prev values

In [145]:
df_kargo_val_count_sort = df_kargo_val_count.sort_values(by=['Ürün Adı', 'Kargodan alınan desi'], ascending=False)  # Not: bir sonraki idmax en büyük ilk değeri aldığından ascending son değer için.   
df_kargo_val_count_sort.reset_index(drop=True, inplace=True)                                            
df_kargo_val_count_sort

,Ürün Adı,Kargodan alınan desi,count
0,Şönil Kilim Desenli Modern Halı (MAKİNADA YIKA...,6.0,1
1,Şönil Kilim Desenli Modern Halı (MAKİNADA YIKA...,5.0,3
2,Şönil Kilim Desenli Modern Halı (MAKİNADA YIKA...,4.0,4
3,Şönil Kilim Desenli Modern Halı (MAKİNADA YIKA...,3.0,1
4,Şönil Kilim Desenli Modern Halı (MAKİNADA YIKA...,10.0,3
...,...,...,...
5203,%100 Doğal Sarı Kantaron Yağı 50 ml TYCCV4VKIN...,9.0,1
5204,%100 Doğal Sarı Kantaron Yağı 50 ml TYCCV4VKIN...,2.0,2
5205,"%100 Doğal Keçi Kılı Yüz Bakım Fırçası, Yüz Ba...",5.0,1
5206,"%100 Doğal Keçi Kılı Yüz Bakım Fırçası, Yüz Ba...",2.0,2


In [146]:
idx = df_kargo_val_count_sort.groupby('Ürün Adı')['count'].idxmax()
idx

Ürün Adı
%100 Doğal Keçi Kılı Yüz Bakım Fırçası, Yüz Bakım Fırçası 1 Adet Isim Yazdırılabilir 002975, one size          5206
%100 Doğal Sarı Kantaron Yağı 50 ml TYCCV4VKIN170056964678741, one size                                        5204
%100 Doğal Soğuk Sıkım Sabit Çörekotu Yağı, Black Cumin Oil, 50 ml 002483, one size                            5202
%100 Doğal Çam Terebentin Yağı, Pine Turpentine Oil, 50 ml 002482, one size                                    5201
0-3 Yaş Zeka Kartları Dokun Hisset Tanı Eğitim Kartları, Yavru Hayvanlar circle toys dokun hisset, one size    5200
                                                                                                               ... 
Şişme Tabanlı 3 Boğumlu Sunset Havuz 114x25 cm 57412, one size                                                    9
Şişme Tabanlı 3 Boğumlu Sunset Havuz 114x25 cm 57412. 57412pompa, one size                                        6
Şönil Kilim Desenli Modern Halı (MAKİNADA YIKANABİLİR SÜNGERLİ 

In [147]:
df_kargo_desi_most_freq = df_kargo_val_count_sort.loc[idx]
df_kargo_desi_most_freq.reset_index(drop=True, inplace=True)
df_kargo_desi_most_freq = df_kargo_desi_most_freq.rename(columns={"Kargodan alınan desi":"kargo desi en sık"})
df_kargo_desi_most_freq

,Ürün Adı,kargo desi en sık,count
0,"%100 Doğal Keçi Kılı Yüz Bakım Fırçası, Yüz Ba...",2.0,2
1,%100 Doğal Sarı Kantaron Yağı 50 ml TYCCV4VKIN...,2.0,2
2,"%100 Doğal Soğuk Sıkım Sabit Çörekotu Yağı, Bl...",1.0,1
3,"%100 Doğal Çam Terebentin Yağı, Pine Turpentin...",2.0,1
4,0-3 Yaş Zeka Kartları Dokun Hisset Tanı Eğitim...,1.0,2
...,...,...,...
1383,Şişme Tabanlı 3 Boğumlu Sunset Havuz 114x25 cm...,2.0,37
1384,Şişme Tabanlı 3 Boğumlu Sunset Havuz 114x25 cm...,2.0,1
1385,Şönil Kilim Desenli Modern Halı (MAKİNADA YIKA...,3.0,1
1386,Şönil Kilim Desenli Modern Halı (MAKİNADA YIKA...,10.0,3


In [148]:
# temp
set1 = set(df_kargo_desi_most_freq["Ürün Adı"])

###### Mean

In [149]:
df_kargo_desi_mean = df_kargo3.groupby("Ürün Adı")[["Kargodan alınan desi"]].mean()
df_kargo_desi_mean.reset_index(inplace=True)
df_kargo_desi_mean.rename(columns={"Kargodan alınan desi":"kargo desi ortalama"}, inplace=True)
df_kargo_desi_mean["kargo desi ortalama"] = df_kargo_desi_mean["kargo desi ortalama"].apply(lambda x: round(x,1))
df_kargo_desi_mean

,Ürün Adı,kargo desi ortalama
0,"%100 Doğal Keçi Kılı Yüz Bakım Fırçası, Yüz Ba...",2.2
1,%100 Doğal Sarı Kantaron Yağı 50 ml TYCCV4VKIN...,4.3
2,"%100 Doğal Soğuk Sıkım Sabit Çörekotu Yağı, Bl...",1.0
3,"%100 Doğal Çam Terebentin Yağı, Pine Turpentin...",2.0
4,0-3 Yaş Zeka Kartları Dokun Hisset Tanı Eğitim...,4.0
...,...,...
1383,Şişme Tabanlı 3 Boğumlu Sunset Havuz 114x25 cm...,2.2
1384,Şişme Tabanlı 3 Boğumlu Sunset Havuz 114x25 cm...,2.0
1385,Şönil Kilim Desenli Modern Halı (MAKİNADA YIKA...,3.0
1386,Şönil Kilim Desenli Modern Halı (MAKİNADA YIKA...,10.0


In [150]:
# temp
set2 = set(df_kargo_desi_mean["Ürün Adı"])

In [151]:
# temp
diff_list = list(set2.difference(set1))
diff_list

[]

In [152]:
# temp
df_kargo3[df_kargo3["Ürün Adı"].isin(diff_list)]

,Barkod,Paket No,Kargo Firması,Sipariş Numarası,İl,İlçe,Ürün Adı,Komisyon Oranı,Marka,Adet,Birim Fiyatı,Satış Tutarı,İndirim Tutarı,Trendyol İndirim Tutarı,Faturalanacak Tutar,Kargodan alınan desi,Faturalanan Kargo Tutarı


###### Max

In [153]:
df_kargo_desi_max = df_kargo3.groupby("Ürün Adı")[["Kargodan alınan desi"]].max()
df_kargo_desi_max.reset_index(inplace=True)
df_kargo_desi_max.rename(columns={"Kargodan alınan desi":"kargo desi max"}, inplace=True)
df_kargo_desi_max

,Ürün Adı,kargo desi max
0,"%100 Doğal Keçi Kılı Yüz Bakım Fırçası, Yüz Ba...",5.0
1,%100 Doğal Sarı Kantaron Yağı 50 ml TYCCV4VKIN...,9.0
2,"%100 Doğal Soğuk Sıkım Sabit Çörekotu Yağı, Bl...",1.0
3,"%100 Doğal Çam Terebentin Yağı, Pine Turpentin...",2.0
4,0-3 Yaş Zeka Kartları Dokun Hisset Tanı Eğitim...,12.0
...,...,...
1383,Şişme Tabanlı 3 Boğumlu Sunset Havuz 114x25 cm...,9.0
1384,Şişme Tabanlı 3 Boğumlu Sunset Havuz 114x25 cm...,2.0
1385,Şönil Kilim Desenli Modern Halı (MAKİNADA YIKA...,3.0
1386,Şönil Kilim Desenli Modern Halı (MAKİNADA YIKA...,10.0


###### Min

In [154]:
df_kargo_desi_min = df_kargo3.groupby("Ürün Adı")[["Kargodan alınan desi"]].min()
df_kargo_desi_min.reset_index(inplace=True)
df_kargo_desi_min.rename(columns={"Kargodan alınan desi":"kargo desi min"}, inplace=True)
df_kargo_desi_min

,Ürün Adı,kargo desi min
0,"%100 Doğal Keçi Kılı Yüz Bakım Fırçası, Yüz Ba...",1.0
1,%100 Doğal Sarı Kantaron Yağı 50 ml TYCCV4VKIN...,2.0
2,"%100 Doğal Soğuk Sıkım Sabit Çörekotu Yağı, Bl...",1.0
3,"%100 Doğal Çam Terebentin Yağı, Pine Turpentin...",2.0
4,0-3 Yaş Zeka Kartları Dokun Hisset Tanı Eğitim...,1.0
...,...,...
1383,Şişme Tabanlı 3 Boğumlu Sunset Havuz 114x25 cm...,1.0
1384,Şişme Tabanlı 3 Boğumlu Sunset Havuz 114x25 cm...,2.0
1385,Şönil Kilim Desenli Modern Halı (MAKİNADA YIKA...,3.0
1386,Şönil Kilim Desenli Modern Halı (MAKİNADA YIKA...,10.0


In [155]:
df_kargo4 = df_kargo3.sort_values(by="Barkod")
df_kargo4 = df_kargo4.reset_index(drop=True)
df_kargo4.head(5)

,Barkod,Paket No,Kargo Firması,Sipariş Numarası,İl,İlçe,Ürün Adı,Komisyon Oranı,Marka,Adet,Birim Fiyatı,Satış Tutarı,İndirim Tutarı,Trendyol İndirim Tutarı,Faturalanacak Tutar,Kargodan alınan desi,Faturalanan Kargo Tutarı
0,000416648254,2280482994,Trendyol Express Marketplace,9352539467,İzmir,Gaziemir,"Pembe Ahşap Kaydıraklı Oyun Evi, Ahşap Oyun Ev...",13.5,ROZİTOYS,1,1900.0,1900.0,10.0,0.0,1890.0,9.0,Henüz fatura kesilmemiştir.
1,000416648254,2200142600,Trendyol Express Marketplace,9261352113,Denizli,Merkezefendi,"Pembe Ahşap Kaydıraklı Oyun Evi, Ahşap Oyun Ev...",13.5,ROZİTOYS,1,1900.0,1900.0,0.0,0.0,1900.0,10.0,103.37
2,000416648254,2288174732,Trendyol Express Marketplace,9361375954,Mersin,Toroslar,"Pembe Ahşap Kaydıraklı Oyun Evi, Ahşap Oyun Ev...",13.5,ROZİTOYS,1,2350.0,2350.0,0.0,0.0,2350.0,10.0,Henüz fatura kesilmemiştir.
3,000416648254,2166714797,Trendyol Express Marketplace,9223437644,İstanbul,Fatih,"Pembe Ahşap Kaydıraklı Oyun Evi, Ahşap Oyun Ev...",13.5,ROZİTOYS,1,1850.0,1850.0,0.0,0.0,1850.0,26.0,200.40
4,000416648254,2227011001,Trendyol Express Marketplace,9291384695,İstanbul,Küçükçekmece,"Pembe Ahşap Kaydıraklı Oyun Evi, Ahşap Oyun Ev...",11.3,ROZİTOYS,1,1800.0,1800.0,0.0,0.0,1800.0,27.0,206.58


In [156]:
df_kargo4 = df_kargo4.drop_duplicates(subset=["Barkod","Ürün Adı","Kargo Firması"])
df_kargo4.reset_index(drop=True, inplace=True)
df_kargo4.head(5)

,Barkod,Paket No,Kargo Firması,Sipariş Numarası,İl,İlçe,Ürün Adı,Komisyon Oranı,Marka,Adet,Birim Fiyatı,Satış Tutarı,İndirim Tutarı,Trendyol İndirim Tutarı,Faturalanacak Tutar,Kargodan alınan desi,Faturalanan Kargo Tutarı
0,000416648254,2280482994,Trendyol Express Marketplace,9352539467,İzmir,Gaziemir,"Pembe Ahşap Kaydıraklı Oyun Evi, Ahşap Oyun Ev...",13.50,ROZİTOYS,1,1900.00,1900.00,10.0,0.0,1890.00,9.0,Henüz fatura kesilmemiştir.
1,000437943412,2219946411,Trendyol Express Marketplace,9283284388,İstanbul,Bakırköy,Çamaşır Kurutma Makinası İçin Uçucu Yağlar Set...,12.20,proclis,1,270.00,270.00,0.0,0.0,270.00,1.0,53.34
2,0006941057446691,2082678312,Trendyol Express Marketplace,9130023184,Ankara,Yenimahalle,Hayvanlı Kumsuz Hacıyatmaz Kaplan 98 Cm X 44 C...,13.73,Intex,1,285.00,285.00,0.0,0.0,285.00,1.0,53.34
3,0006941057446691,2176825666,MNG Kargo Marketplace,9234601826,Eskişehir,Alpu,Hayvanlı Kumsuz Hacıyatmaz Kaplan 98 Cm X 44 C...,13.73,Intex,1,285.00,285.00,0.0,0.0,285.00,2.0,58.79
4,002154600346,2116926027,Trendyol Express Marketplace,9167602070,İstanbul,Başakşehir,"Dekoratif Salyangoz Jüt Dokuma Bardak Altlığı,...",19.32,Pikas,1,249.99,249.99,0.0,0.0,249.99,8.0,94.06


In [157]:
df_kargo_desi_min_merge = pd.merge(df_kargo4, df_kargo_desi_min, how="left", on="Ürün Adı")
df_kargo_desi_min_merge.head(5)

,Barkod,Paket No,Kargo Firması,Sipariş Numarası,İl,İlçe,Ürün Adı,Komisyon Oranı,Marka,Adet,Birim Fiyatı,Satış Tutarı,İndirim Tutarı,Trendyol İndirim Tutarı,Faturalanacak Tutar,Kargodan alınan desi,Faturalanan Kargo Tutarı,kargo desi min
0,000416648254,2280482994,Trendyol Express Marketplace,9352539467,İzmir,Gaziemir,"Pembe Ahşap Kaydıraklı Oyun Evi, Ahşap Oyun Ev...",13.50,ROZİTOYS,1,1900.00,1900.00,10.0,0.0,1890.00,9.0,Henüz fatura kesilmemiştir.,9.0
1,000437943412,2219946411,Trendyol Express Marketplace,9283284388,İstanbul,Bakırköy,Çamaşır Kurutma Makinası İçin Uçucu Yağlar Set...,12.20,proclis,1,270.00,270.00,0.0,0.0,270.00,1.0,53.34,1.0
2,0006941057446691,2082678312,Trendyol Express Marketplace,9130023184,Ankara,Yenimahalle,Hayvanlı Kumsuz Hacıyatmaz Kaplan 98 Cm X 44 C...,13.73,Intex,1,285.00,285.00,0.0,0.0,285.00,1.0,53.34,1.0
3,0006941057446691,2176825666,MNG Kargo Marketplace,9234601826,Eskişehir,Alpu,Hayvanlı Kumsuz Hacıyatmaz Kaplan 98 Cm X 44 C...,13.73,Intex,1,285.00,285.00,0.0,0.0,285.00,2.0,58.79,1.0
4,002154600346,2116926027,Trendyol Express Marketplace,9167602070,İstanbul,Başakşehir,"Dekoratif Salyangoz Jüt Dokuma Bardak Altlığı,...",19.32,Pikas,1,249.99,249.99,0.0,0.0,249.99,8.0,94.06,8.0


In [158]:
df_kargo_desi_min_max_merge = pd.merge(df_kargo_desi_min_merge, df_kargo_desi_max, how="left", on="Ürün Adı")
df_kargo_desi_min_max_merge.head(5)

,Barkod,Paket No,Kargo Firması,Sipariş Numarası,İl,İlçe,Ürün Adı,Komisyon Oranı,Marka,Adet,Birim Fiyatı,Satış Tutarı,İndirim Tutarı,Trendyol İndirim Tutarı,Faturalanacak Tutar,Kargodan alınan desi,Faturalanan Kargo Tutarı,kargo desi min,kargo desi max
0,000416648254,2280482994,Trendyol Express Marketplace,9352539467,İzmir,Gaziemir,"Pembe Ahşap Kaydıraklı Oyun Evi, Ahşap Oyun Ev...",13.50,ROZİTOYS,1,1900.00,1900.00,10.0,0.0,1890.00,9.0,Henüz fatura kesilmemiştir.,9.0,27.0
1,000437943412,2219946411,Trendyol Express Marketplace,9283284388,İstanbul,Bakırköy,Çamaşır Kurutma Makinası İçin Uçucu Yağlar Set...,12.20,proclis,1,270.00,270.00,0.0,0.0,270.00,1.0,53.34,1.0,2.0
2,0006941057446691,2082678312,Trendyol Express Marketplace,9130023184,Ankara,Yenimahalle,Hayvanlı Kumsuz Hacıyatmaz Kaplan 98 Cm X 44 C...,13.73,Intex,1,285.00,285.00,0.0,0.0,285.00,1.0,53.34,1.0,2.0
3,0006941057446691,2176825666,MNG Kargo Marketplace,9234601826,Eskişehir,Alpu,Hayvanlı Kumsuz Hacıyatmaz Kaplan 98 Cm X 44 C...,13.73,Intex,1,285.00,285.00,0.0,0.0,285.00,2.0,58.79,1.0,2.0
4,002154600346,2116926027,Trendyol Express Marketplace,9167602070,İstanbul,Başakşehir,"Dekoratif Salyangoz Jüt Dokuma Bardak Altlığı,...",19.32,Pikas,1,249.99,249.99,0.0,0.0,249.99,8.0,94.06,8.0,8.0


In [159]:
df_kargo_desi_min_max_mean_merge = pd.merge(df_kargo_desi_min_max_merge, df_kargo_desi_mean, how="left", on="Ürün Adı")
df_kargo_desi_min_max_mean_merge.head(5)

,Barkod,Paket No,Kargo Firması,Sipariş Numarası,İl,İlçe,Ürün Adı,Komisyon Oranı,Marka,Adet,Birim Fiyatı,Satış Tutarı,İndirim Tutarı,Trendyol İndirim Tutarı,Faturalanacak Tutar,Kargodan alınan desi,Faturalanan Kargo Tutarı,kargo desi min,kargo desi max,kargo desi ortalama
0,000416648254,2280482994,Trendyol Express Marketplace,9352539467,İzmir,Gaziemir,"Pembe Ahşap Kaydıraklı Oyun Evi, Ahşap Oyun Ev...",13.50,ROZİTOYS,1,1900.00,1900.00,10.0,0.0,1890.00,9.0,Henüz fatura kesilmemiştir.,9.0,27.0,20.0
1,000437943412,2219946411,Trendyol Express Marketplace,9283284388,İstanbul,Bakırköy,Çamaşır Kurutma Makinası İçin Uçucu Yağlar Set...,12.20,proclis,1,270.00,270.00,0.0,0.0,270.00,1.0,53.34,1.0,2.0,1.6
2,0006941057446691,2082678312,Trendyol Express Marketplace,9130023184,Ankara,Yenimahalle,Hayvanlı Kumsuz Hacıyatmaz Kaplan 98 Cm X 44 C...,13.73,Intex,1,285.00,285.00,0.0,0.0,285.00,1.0,53.34,1.0,2.0,1.3
3,0006941057446691,2176825666,MNG Kargo Marketplace,9234601826,Eskişehir,Alpu,Hayvanlı Kumsuz Hacıyatmaz Kaplan 98 Cm X 44 C...,13.73,Intex,1,285.00,285.00,0.0,0.0,285.00,2.0,58.79,1.0,2.0,1.3
4,002154600346,2116926027,Trendyol Express Marketplace,9167602070,İstanbul,Başakşehir,"Dekoratif Salyangoz Jüt Dokuma Bardak Altlığı,...",19.32,Pikas,1,249.99,249.99,0.0,0.0,249.99,8.0,94.06,8.0,8.0,8.0


In [160]:
# df_kargo_desi_min_max_mean_merge.to_excel(r"C:\Users\user\Downloads\Trendyol Kargo Desi Min Max Ortalama.xlsx", index=False)

In [161]:
df_kargo_desi_min_max_mean_most_merge = pd.merge(df_kargo_desi_min_max_mean_merge, df_kargo_desi_most_freq, how="left", on="Ürün Adı")
df_kargo_desi_min_max_mean_most_merge.head(5)

,Barkod,Paket No,Kargo Firması,Sipariş Numarası,İl,İlçe,Ürün Adı,Komisyon Oranı,Marka,Adet,...,İndirim Tutarı,Trendyol İndirim Tutarı,Faturalanacak Tutar,Kargodan alınan desi,Faturalanan Kargo Tutarı,kargo desi min,kargo desi max,kargo desi ortalama,kargo desi en sık,count
0,000416648254,2280482994,Trendyol Express Marketplace,9352539467,İzmir,Gaziemir,"Pembe Ahşap Kaydıraklı Oyun Evi, Ahşap Oyun Ev...",13.50,ROZİTOYS,1,...,10.0,0.0,1890.00,9.0,Henüz fatura kesilmemiştir.,9.0,27.0,20.0,27.0,2
1,000437943412,2219946411,Trendyol Express Marketplace,9283284388,İstanbul,Bakırköy,Çamaşır Kurutma Makinası İçin Uçucu Yağlar Set...,12.20,proclis,1,...,0.0,0.0,270.00,1.0,53.34,1.0,2.0,1.6,2.0,3
2,0006941057446691,2082678312,Trendyol Express Marketplace,9130023184,Ankara,Yenimahalle,Hayvanlı Kumsuz Hacıyatmaz Kaplan 98 Cm X 44 C...,13.73,Intex,1,...,0.0,0.0,285.00,1.0,53.34,1.0,2.0,1.3,1.0,2
3,0006941057446691,2176825666,MNG Kargo Marketplace,9234601826,Eskişehir,Alpu,Hayvanlı Kumsuz Hacıyatmaz Kaplan 98 Cm X 44 C...,13.73,Intex,1,...,0.0,0.0,285.00,2.0,58.79,1.0,2.0,1.3,1.0,2
4,002154600346,2116926027,Trendyol Express Marketplace,9167602070,İstanbul,Başakşehir,"Dekoratif Salyangoz Jüt Dokuma Bardak Altlığı,...",19.32,Pikas,1,...,0.0,0.0,249.99,8.0,94.06,8.0,8.0,8.0,8.0,1


In [162]:
# df_kargo_desi_min_max_mean_most_merge.to_excel(r"C:\Users\user\Downloads\Trendyol Kargo Desi Min Max Ortalama En Sık.xlsx", index=False)
# df_kargo_desi_min_max_mean_most_merge.to_excel(r"C:\Users\user\Desktop\Data Analysis\Adoba\Sales_Report\Result\Trendyol\Trendyol Kargo Desi Min Max Ortalama En Sık.xlsx", index=False)

#### Kargo Desi Fiyat Ekleme

In [163]:
df_kargo_desi_fiyat = pd.read_excel(r"C:\Users\user\Desktop\Data Analysis\Adoba\Sales_Report\Data\Kargo Price\29.07.2024\Kargo Desi Fiyatları (29.07.2024).xlsx")
df_kargo_desi_fiyat

,Desi/KG,Aras,MNG,PTT,Sendeo,Sürat,TEX,Yurtiçi,Borusan,CEVA,Horoz
0,0,56.988,62.388,57.264,59.160,58.464,53.340,75.972,392.412,496.524,464.256
1,1,56.988,62.388,57.264,59.160,58.464,53.340,75.972,392.412,496.524,464.256
2,2,61.284,62.388,57.264,59.160,58.464,53.340,75.972,392.412,496.524,464.256
3,3,68.652,68.388,70.896,70.452,67.692,62.640,81.216,392.412,496.524,464.256
4,4,74.832,75.588,70.896,72.852,74.400,67.128,82.968,392.412,496.524,464.256
...,...,...,...,...,...,...,...,...,...,...,...
96,96,729.864,1535.196,1264.380,661.044,842.112,691.956,909.120,1234.176,1115.028,1019.484
97,97,737.208,1553.184,1276.248,666.516,850.584,698.796,918.132,1247.028,1126.704,1030.104
98,98,744.552,1571.172,1288.128,672.000,859.080,705.648,927.144,1259.880,1138.260,1040.724
99,99,751.896,1589.160,1299.996,677.472,867.540,712.488,936.156,1272.744,1149.936,1051.332


In [164]:
df_kargo_desi_fiyat.columns

Index(['Desi/KG', 'Aras', 'MNG', 'PTT', 'Sendeo', 'Sürat', 'TEX', 'Yurtiçi',
       'Borusan', 'CEVA', 'Horoz'],
      dtype='object')

In [165]:
# kargo_list = ['Aras', 'MNG', 'PTT', 'Sendeo', 'Sürat', 'TEX', 'Yurtiçi','Borusan', 'CEVA', 'Horoz']

In [166]:
# for kargo in kargo_list:
#     df_kargo_desi_fiyat2[f"{kargo}"] = df_kargo_desi_fiyat2[f"{kargo}"]*1.20
# df_kargo_desi_fiyat2

In [167]:
df_kargo_desi_fiyat2 = pd.melt(df_kargo_desi_fiyat, id_vars=['Desi/KG'], value_vars=['Aras','MNG','PTT','Sendeo','Sürat','TEX','Yurtiçi','Borusan','CEVA','Horoz'])
df_kargo_desi_fiyat2

,Desi/KG,variable,value
0,0,Aras,56.988
1,1,Aras,56.988
2,2,Aras,61.284
3,3,Aras,68.652
4,4,Aras,74.832
...,...,...,...
1005,96,Horoz,1019.484
1006,97,Horoz,1030.104
1007,98,Horoz,1040.724
1008,99,Horoz,1051.332


In [168]:
df_kargo_desi_fiyat2 = df_kargo_desi_fiyat2.rename(columns={"variable":"Kargo Firması","value":"Kargo Fiyat"})
df_kargo_desi_fiyat2

,Desi/KG,Kargo Firması,Kargo Fiyat
0,0,Aras,56.988
1,1,Aras,56.988
2,2,Aras,61.284
3,3,Aras,68.652
4,4,Aras,74.832
...,...,...,...
1005,96,Horoz,1019.484
1006,97,Horoz,1030.104
1007,98,Horoz,1040.724
1008,99,Horoz,1051.332


In [169]:
df_trend_kargo = pd.read_excel(r"C:\Users\user\Desktop\Data Analysis\Adoba\Sales_Report\Result\Trendyol\Kargo Desi Fiyat\Trendyol Kargo Desi Min Max Ortalama En Sık.xlsx")
df_trend_kargo.head(5)

,Barkod,Paket No,Kargo Firması,Sipariş Numarası,İl,İlçe,Ürün Adı,Komisyon Oranı,Marka,Adet,...,İndirim Tutarı,Trendyol İndirim Tutarı,Faturalanacak Tutar,Kargodan alınan desi,Faturalanan Kargo Tutarı,kargo desi min,kargo desi max,kargo desi ortalama,kargo desi en sık,count
0,000416648254,2280482994,Trendyol Express Marketplace,9352539467,İzmir,Gaziemir,"Pembe Ahşap Kaydıraklı Oyun Evi, Ahşap Oyun Ev...",13.50,ROZİTOYS,1,...,10.0,0,1890.00,9.0,Henüz fatura kesilmemiştir.,9.0,27.0,20.0,27.0,2
1,000437943412,2219946411,Trendyol Express Marketplace,9283284388,İstanbul,Bakırköy,Çamaşır Kurutma Makinası İçin Uçucu Yağlar Set...,12.20,proclis,1,...,0.0,0,270.00,1.0,53.34,1.0,2.0,1.6,2.0,3
2,0006941057446691,2082678312,Trendyol Express Marketplace,9130023184,Ankara,Yenimahalle,Hayvanlı Kumsuz Hacıyatmaz Kaplan 98 Cm X 44 C...,13.73,Intex,1,...,0.0,0,285.00,1.0,53.34,1.0,2.0,1.3,1.0,2
3,0006941057446691,2176825666,MNG Kargo Marketplace,9234601826,Eskişehir,Alpu,Hayvanlı Kumsuz Hacıyatmaz Kaplan 98 Cm X 44 C...,13.73,Intex,1,...,0.0,0,285.00,2.0,58.79,1.0,2.0,1.3,1.0,2
4,002154600346,2116926027,Trendyol Express Marketplace,9167602070,İstanbul,Başakşehir,"Dekoratif Salyangoz Jüt Dokuma Bardak Altlığı,...",19.32,Pikas,1,...,0.0,0,249.99,8.0,94.06,8.0,8.0,8.0,8.0,1


In [170]:
df_trend_kargo["Kargo Firması"].unique()

array(['Trendyol Express Marketplace', 'MNG Kargo Marketplace',
       'Yurtiçi Kargo Marketplace', 'PTT Kargo Marketplace',
       'Sürat Kargo Marketplace'], dtype=object)

In [171]:
def change_cargo_name(var):
    '''
    This function converts cargo names.
    '''
    if var == "Trendyol Express Marketplace":
        return 'TEX'
    elif var == "MNG Kargo Marketplace":
        return 'MNG'
    elif var == 'Yurtiçi Kargo Marketplace':
        return 'Yurtiçi'
    elif var == 'PTT Kargo Marketplace':
        return 'PTT'
    elif var == 'Sürat Kargo Marketplace':
        return 'Sürat'
    else:
        return var

In [172]:
df_trend_kargo["Kargo Firması"] = df_trend_kargo["Kargo Firması"].apply(lambda var: change_cargo_name(var))
df_trend_kargo.head(5)

,Barkod,Paket No,Kargo Firması,Sipariş Numarası,İl,İlçe,Ürün Adı,Komisyon Oranı,Marka,Adet,...,İndirim Tutarı,Trendyol İndirim Tutarı,Faturalanacak Tutar,Kargodan alınan desi,Faturalanan Kargo Tutarı,kargo desi min,kargo desi max,kargo desi ortalama,kargo desi en sık,count
0,000416648254,2280482994,TEX,9352539467,İzmir,Gaziemir,"Pembe Ahşap Kaydıraklı Oyun Evi, Ahşap Oyun Ev...",13.50,ROZİTOYS,1,...,10.0,0,1890.00,9.0,Henüz fatura kesilmemiştir.,9.0,27.0,20.0,27.0,2
1,000437943412,2219946411,TEX,9283284388,İstanbul,Bakırköy,Çamaşır Kurutma Makinası İçin Uçucu Yağlar Set...,12.20,proclis,1,...,0.0,0,270.00,1.0,53.34,1.0,2.0,1.6,2.0,3
2,0006941057446691,2082678312,TEX,9130023184,Ankara,Yenimahalle,Hayvanlı Kumsuz Hacıyatmaz Kaplan 98 Cm X 44 C...,13.73,Intex,1,...,0.0,0,285.00,1.0,53.34,1.0,2.0,1.3,1.0,2
3,0006941057446691,2176825666,MNG,9234601826,Eskişehir,Alpu,Hayvanlı Kumsuz Hacıyatmaz Kaplan 98 Cm X 44 C...,13.73,Intex,1,...,0.0,0,285.00,2.0,58.79,1.0,2.0,1.3,1.0,2
4,002154600346,2116926027,TEX,9167602070,İstanbul,Başakşehir,"Dekoratif Salyangoz Jüt Dokuma Bardak Altlığı,...",19.32,Pikas,1,...,0.0,0,249.99,8.0,94.06,8.0,8.0,8.0,8.0,1


In [173]:
df_trend_kargo2 = df_trend_kargo.dropna(subset="Kargodan alınan desi")
df_trend_kargo2.head(5)

,Barkod,Paket No,Kargo Firması,Sipariş Numarası,İl,İlçe,Ürün Adı,Komisyon Oranı,Marka,Adet,...,İndirim Tutarı,Trendyol İndirim Tutarı,Faturalanacak Tutar,Kargodan alınan desi,Faturalanan Kargo Tutarı,kargo desi min,kargo desi max,kargo desi ortalama,kargo desi en sık,count
0,000416648254,2280482994,TEX,9352539467,İzmir,Gaziemir,"Pembe Ahşap Kaydıraklı Oyun Evi, Ahşap Oyun Ev...",13.50,ROZİTOYS,1,...,10.0,0,1890.00,9.0,Henüz fatura kesilmemiştir.,9.0,27.0,20.0,27.0,2
1,000437943412,2219946411,TEX,9283284388,İstanbul,Bakırköy,Çamaşır Kurutma Makinası İçin Uçucu Yağlar Set...,12.20,proclis,1,...,0.0,0,270.00,1.0,53.34,1.0,2.0,1.6,2.0,3
2,0006941057446691,2082678312,TEX,9130023184,Ankara,Yenimahalle,Hayvanlı Kumsuz Hacıyatmaz Kaplan 98 Cm X 44 C...,13.73,Intex,1,...,0.0,0,285.00,1.0,53.34,1.0,2.0,1.3,1.0,2
3,0006941057446691,2176825666,MNG,9234601826,Eskişehir,Alpu,Hayvanlı Kumsuz Hacıyatmaz Kaplan 98 Cm X 44 C...,13.73,Intex,1,...,0.0,0,285.00,2.0,58.79,1.0,2.0,1.3,1.0,2
4,002154600346,2116926027,TEX,9167602070,İstanbul,Başakşehir,"Dekoratif Salyangoz Jüt Dokuma Bardak Altlığı,...",19.32,Pikas,1,...,0.0,0,249.99,8.0,94.06,8.0,8.0,8.0,8.0,1


In [174]:
math.ceil(4)

4

In [175]:
df_trend_kargo2.loc[:,"kargo desi min"] = df_trend_kargo2.loc[:,"kargo desi min"].apply(lambda x: math.ceil(x))
df_trend_kargo2.loc[:,"kargo desi max"] = df_trend_kargo2.loc[:,"kargo desi max"].apply(lambda x: math.ceil(x))
df_trend_kargo2.loc[:,"kargo desi ortalama"] = df_trend_kargo2.loc[:,"kargo desi ortalama"].apply(lambda x: math.ceil(x))
df_trend_kargo2.loc[:,"kargo desi en sık"] = df_trend_kargo2.loc[:,"kargo desi en sık"].apply(lambda x: math.ceil(x))
df_trend_kargo2.reset_index(drop=True, inplace=True)
df_trend_kargo2.head(5)

,Barkod,Paket No,Kargo Firması,Sipariş Numarası,İl,İlçe,Ürün Adı,Komisyon Oranı,Marka,Adet,...,İndirim Tutarı,Trendyol İndirim Tutarı,Faturalanacak Tutar,Kargodan alınan desi,Faturalanan Kargo Tutarı,kargo desi min,kargo desi max,kargo desi ortalama,kargo desi en sık,count
0,000416648254,2280482994,TEX,9352539467,İzmir,Gaziemir,"Pembe Ahşap Kaydıraklı Oyun Evi, Ahşap Oyun Ev...",13.50,ROZİTOYS,1,...,10.0,0,1890.00,9.0,Henüz fatura kesilmemiştir.,9.0,27.0,20.0,27.0,2
1,000437943412,2219946411,TEX,9283284388,İstanbul,Bakırköy,Çamaşır Kurutma Makinası İçin Uçucu Yağlar Set...,12.20,proclis,1,...,0.0,0,270.00,1.0,53.34,1.0,2.0,2.0,2.0,3
2,0006941057446691,2082678312,TEX,9130023184,Ankara,Yenimahalle,Hayvanlı Kumsuz Hacıyatmaz Kaplan 98 Cm X 44 C...,13.73,Intex,1,...,0.0,0,285.00,1.0,53.34,1.0,2.0,2.0,1.0,2
3,0006941057446691,2176825666,MNG,9234601826,Eskişehir,Alpu,Hayvanlı Kumsuz Hacıyatmaz Kaplan 98 Cm X 44 C...,13.73,Intex,1,...,0.0,0,285.00,2.0,58.79,1.0,2.0,2.0,1.0,2
4,002154600346,2116926027,TEX,9167602070,İstanbul,Başakşehir,"Dekoratif Salyangoz Jüt Dokuma Bardak Altlığı,...",19.32,Pikas,1,...,0.0,0,249.99,8.0,94.06,8.0,8.0,8.0,8.0,1


In [176]:
df_kargo_desi_fiyat2.head(5)

,Desi/KG,Kargo Firması,Kargo Fiyat
0,0,Aras,56.988
1,1,Aras,56.988
2,2,Aras,61.284
3,3,Aras,68.652
4,4,Aras,74.832


In [177]:
# 1 way cargo price
df_trend_kargo_desi_min = pd.merge(df_trend_kargo2, df_kargo_desi_fiyat2, how='left', left_on=['kargo desi min','Kargo Firması'], right_on=['Desi/KG','Kargo Firması']).drop('Desi/KG', axis=1)
df_trend_kargo_desi_min.rename(columns={"Kargo Fiyat":"Kargo Fiyat Min"}, inplace=True)
df_trend_kargo_desi_min.head(5)

,Barkod,Paket No,Kargo Firması,Sipariş Numarası,İl,İlçe,Ürün Adı,Komisyon Oranı,Marka,Adet,...,Trendyol İndirim Tutarı,Faturalanacak Tutar,Kargodan alınan desi,Faturalanan Kargo Tutarı,kargo desi min,kargo desi max,kargo desi ortalama,kargo desi en sık,count,Kargo Fiyat Min
0,000416648254,2280482994,TEX,9352539467,İzmir,Gaziemir,"Pembe Ahşap Kaydıraklı Oyun Evi, Ahşap Oyun Ev...",13.50,ROZİTOYS,1,...,0,1890.00,9.0,Henüz fatura kesilmemiştir.,9.0,27.0,20.0,27.0,2,97.620
1,000437943412,2219946411,TEX,9283284388,İstanbul,Bakırköy,Çamaşır Kurutma Makinası İçin Uçucu Yağlar Set...,12.20,proclis,1,...,0,270.00,1.0,53.34,1.0,2.0,2.0,2.0,3,53.340
2,0006941057446691,2082678312,TEX,9130023184,Ankara,Yenimahalle,Hayvanlı Kumsuz Hacıyatmaz Kaplan 98 Cm X 44 C...,13.73,Intex,1,...,0,285.00,1.0,53.34,1.0,2.0,2.0,1.0,2,53.340
3,0006941057446691,2176825666,MNG,9234601826,Eskişehir,Alpu,Hayvanlı Kumsuz Hacıyatmaz Kaplan 98 Cm X 44 C...,13.73,Intex,1,...,0,285.00,2.0,58.79,1.0,2.0,2.0,1.0,2,62.388
4,002154600346,2116926027,TEX,9167602070,İstanbul,Başakşehir,"Dekoratif Salyangoz Jüt Dokuma Bardak Altlığı,...",19.32,Pikas,1,...,0,249.99,8.0,94.06,8.0,8.0,8.0,8.0,1,94.068


In [178]:
df_trend_kargo_desi_min_max = pd.merge(df_trend_kargo_desi_min, df_kargo_desi_fiyat2, how='left', left_on=['kargo desi max','Kargo Firması'], right_on=['Desi/KG','Kargo Firması']).drop('Desi/KG', axis=1)
df_trend_kargo_desi_min_max.rename(columns={"Kargo Fiyat":"Kargo Fiyat Max"}, inplace=True)
df_trend_kargo_desi_min_max.head(5)

,Barkod,Paket No,Kargo Firması,Sipariş Numarası,İl,İlçe,Ürün Adı,Komisyon Oranı,Marka,Adet,...,Faturalanacak Tutar,Kargodan alınan desi,Faturalanan Kargo Tutarı,kargo desi min,kargo desi max,kargo desi ortalama,kargo desi en sık,count,Kargo Fiyat Min,Kargo Fiyat Max
0,000416648254,2280482994,TEX,9352539467,İzmir,Gaziemir,"Pembe Ahşap Kaydıraklı Oyun Evi, Ahşap Oyun Ev...",13.50,ROZİTOYS,1,...,1890.00,9.0,Henüz fatura kesilmemiştir.,9.0,27.0,20.0,27.0,2,97.620,206.580
1,000437943412,2219946411,TEX,9283284388,İstanbul,Bakırköy,Çamaşır Kurutma Makinası İçin Uçucu Yağlar Set...,12.20,proclis,1,...,270.00,1.0,53.34,1.0,2.0,2.0,2.0,3,53.340,53.340
2,0006941057446691,2082678312,TEX,9130023184,Ankara,Yenimahalle,Hayvanlı Kumsuz Hacıyatmaz Kaplan 98 Cm X 44 C...,13.73,Intex,1,...,285.00,1.0,53.34,1.0,2.0,2.0,1.0,2,53.340,53.340
3,0006941057446691,2176825666,MNG,9234601826,Eskişehir,Alpu,Hayvanlı Kumsuz Hacıyatmaz Kaplan 98 Cm X 44 C...,13.73,Intex,1,...,285.00,2.0,58.79,1.0,2.0,2.0,1.0,2,62.388,62.388
4,002154600346,2116926027,TEX,9167602070,İstanbul,Başakşehir,"Dekoratif Salyangoz Jüt Dokuma Bardak Altlığı,...",19.32,Pikas,1,...,249.99,8.0,94.06,8.0,8.0,8.0,8.0,1,94.068,94.068


In [179]:
df_trend_kargo_desi_min_max_mean = pd.merge(df_trend_kargo_desi_min_max, df_kargo_desi_fiyat2, how='left', left_on=['kargo desi ortalama','Kargo Firması'], right_on=['Desi/KG','Kargo Firması']).drop('Desi/KG', axis=1)
df_trend_kargo_desi_min_max_mean.rename(columns={"Kargo Fiyat":"Kargo Fiyat Ortalama"}, inplace=True)
df_trend_kargo_desi_min_max_mean.head(5)

,Barkod,Paket No,Kargo Firması,Sipariş Numarası,İl,İlçe,Ürün Adı,Komisyon Oranı,Marka,Adet,...,Kargodan alınan desi,Faturalanan Kargo Tutarı,kargo desi min,kargo desi max,kargo desi ortalama,kargo desi en sık,count,Kargo Fiyat Min,Kargo Fiyat Max,Kargo Fiyat Ortalama
0,000416648254,2280482994,TEX,9352539467,İzmir,Gaziemir,"Pembe Ahşap Kaydıraklı Oyun Evi, Ahşap Oyun Ev...",13.50,ROZİTOYS,1,...,9.0,Henüz fatura kesilmemiştir.,9.0,27.0,20.0,27.0,2,97.620,206.580,160.488
1,000437943412,2219946411,TEX,9283284388,İstanbul,Bakırköy,Çamaşır Kurutma Makinası İçin Uçucu Yağlar Set...,12.20,proclis,1,...,1.0,53.34,1.0,2.0,2.0,2.0,3,53.340,53.340,53.340
2,0006941057446691,2082678312,TEX,9130023184,Ankara,Yenimahalle,Hayvanlı Kumsuz Hacıyatmaz Kaplan 98 Cm X 44 C...,13.73,Intex,1,...,1.0,53.34,1.0,2.0,2.0,1.0,2,53.340,53.340,53.340
3,0006941057446691,2176825666,MNG,9234601826,Eskişehir,Alpu,Hayvanlı Kumsuz Hacıyatmaz Kaplan 98 Cm X 44 C...,13.73,Intex,1,...,2.0,58.79,1.0,2.0,2.0,1.0,2,62.388,62.388,62.388
4,002154600346,2116926027,TEX,9167602070,İstanbul,Başakşehir,"Dekoratif Salyangoz Jüt Dokuma Bardak Altlığı,...",19.32,Pikas,1,...,8.0,94.06,8.0,8.0,8.0,8.0,1,94.068,94.068,94.068


In [180]:
df_trend_kargo_desi_min_max_mean_most = pd.merge(df_trend_kargo_desi_min_max_mean, df_kargo_desi_fiyat2, how='left', left_on=['kargo desi en sık','Kargo Firması'], right_on=['Desi/KG','Kargo Firması']).drop('Desi/KG', axis=1)
df_trend_kargo_desi_min_max_mean_most.rename(columns={"Kargo Fiyat":"Kargo Fiyat En Sık"}, inplace=True)
df_trend_kargo_desi_min_max_mean_most.head(5)

,Barkod,Paket No,Kargo Firması,Sipariş Numarası,İl,İlçe,Ürün Adı,Komisyon Oranı,Marka,Adet,...,Faturalanan Kargo Tutarı,kargo desi min,kargo desi max,kargo desi ortalama,kargo desi en sık,count,Kargo Fiyat Min,Kargo Fiyat Max,Kargo Fiyat Ortalama,Kargo Fiyat En Sık
0,000416648254,2280482994,TEX,9352539467,İzmir,Gaziemir,"Pembe Ahşap Kaydıraklı Oyun Evi, Ahşap Oyun Ev...",13.50,ROZİTOYS,1,...,Henüz fatura kesilmemiştir.,9.0,27.0,20.0,27.0,2,97.620,206.580,160.488,206.580
1,000437943412,2219946411,TEX,9283284388,İstanbul,Bakırköy,Çamaşır Kurutma Makinası İçin Uçucu Yağlar Set...,12.20,proclis,1,...,53.34,1.0,2.0,2.0,2.0,3,53.340,53.340,53.340,53.340
2,0006941057446691,2082678312,TEX,9130023184,Ankara,Yenimahalle,Hayvanlı Kumsuz Hacıyatmaz Kaplan 98 Cm X 44 C...,13.73,Intex,1,...,53.34,1.0,2.0,2.0,1.0,2,53.340,53.340,53.340,53.340
3,0006941057446691,2176825666,MNG,9234601826,Eskişehir,Alpu,Hayvanlı Kumsuz Hacıyatmaz Kaplan 98 Cm X 44 C...,13.73,Intex,1,...,58.79,1.0,2.0,2.0,1.0,2,62.388,62.388,62.388,62.388
4,002154600346,2116926027,TEX,9167602070,İstanbul,Başakşehir,"Dekoratif Salyangoz Jüt Dokuma Bardak Altlığı,...",19.32,Pikas,1,...,94.06,8.0,8.0,8.0,8.0,1,94.068,94.068,94.068,94.068


In [181]:
# # 2 way cargo price
# df_trend_kargo2.loc[:,"Kargo Fiyat Min"] = df_trend_kargo2.apply(lambda row: df_kargo_desi_fiyat[df_kargo_desi_fiyat.loc[:,"Desi/KG"] == row["kargo desi min"]][f"{row["Kargo Firması"]}"].values[0], axis=1)
# df_trend_kargo2.loc[:,"Kargo Fiyat Max"] = df_trend_kargo2.apply(lambda row: df_kargo_desi_fiyat[df_kargo_desi_fiyat.loc[:,"Desi/KG"] == row["kargo desi max"]][f"{row["Kargo Firması"]}"].values[0], axis=1)
# df_trend_kargo2.loc[:,"Kargo Fiyat Ortalama"] = df_trend_kargo2.apply(lambda row: df_kargo_desi_fiyat[df_kargo_desi_fiyat.loc[:,"Desi/KG"] == row["kargo desi ortalama"]][f"{row["Kargo Firması"]}"].values[0], axis=1)
# df_trend_kargo2.loc[:,"Kargo Fiyat En Sık"] = df_trend_kargo2.apply(lambda row: df_kargo_desi_fiyat[df_kargo_desi_fiyat.loc[:,"Desi/KG"] == row["kargo desi en sık"]][f"{row["Kargo Firması"]}"].values[0], axis=1)
# df_trend_kargo_desi_min_max_mean = df_trend_kargo2
# df_trend_kargo_desi_min_max_mean

In [182]:
# df_trend_kargo_desi_min_max_mean_most.to_excel(r"C:\Users\user\Downloads\Trendyol Kargo Desi Min Max Ortalama En Sık Fiyat.xlsx", index=False)

#### Kargo Diğer Fiyat Ekleme

In [20]:
df_kargo_other = pd.read_excel(r"C:\Users\user\Desktop\Data Analysis\Adoba\Sales_Report\Data\Trendyol\Kargo\30.07.2024\trendyol kargo.xlsx")
df_kargo_other

,Barkod,Paket No,Kargo Firması,Sipariş Numarası,İl,İlçe,Ürün Adı,Komisyon Oranı,Marka,Adet,Birim Fiyatı,Satış Tutarı,İndirim Tutarı,Trendyol İndirim Tutarı,Faturalanacak Tutar,Kargodan alınan desi,Faturalanan Kargo Tutarı
0,97815659247966334,1935755636,Trendyol Express Marketplace,8061482665,Aydın,Germencik,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, Sakl...",20.34,ALİTECH,1,375.00,375.00,0.0,0.0,375.00,9.0,97.62
1,ALS3463102806,1935764998,Trendyol Express Marketplace,2424519644,Adıyaman,Gölbaşı,3'lü Çok Amaçlı Kutu Mega Çizgili Kahverengi 6...,16.20,ALAS,1,350.00,350.00,0.0,0.0,350.00,10.0,103.37
2,604808214197,1935781088,MNG Kargo Marketplace,8061494292,İstanbul,Çekmeköy,"Ahşap Özel İsim Baskılı, Kare Kedi Köpek Mama ...",18.31,GOBYPET,1,149.00,149.00,0.0,0.0,149.00,2.0,35.49
3,ÇG742305202381,1935783700,MNG Kargo Marketplace,2424528751,İstanbul,Kadıköy,"Çg74 Ahşap Çocuk Makyaj Masası Tabureli,unicor...",16.27,ROZİTOYS,1,1200.00,1200.00,0.0,0.0,1200.00,10.0,112.79
4,868407600769,1935789958,MNG Kargo Marketplace,8061497847,İstanbul,Ümraniye,Çömlek Ayaklı Sırlı Seramik Kedi Maması Kabı 2...,18.31,GOBYPET,1,160.00,160.00,0.0,0.0,160.00,4.0,71.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88791,003548003548,2306175520,Trendyol Express Marketplace,9381935071,Tekirdağ,Çerkezköy,"Yüzük Zikirmatik Karışık Renk, Yeni Nesil Ziki...",22.50,Pikas,1,29.99,29.99,0.0,0.0,29.99,NaN,Henüz fatura kesilmemiştir.
88792,8680998014047,2306177666,Trendyol Express Marketplace,9381935705,Bitlis,Adilcevaz,"Rosegold Kafija Kahve Makinesi KAFİJA, one size",11.00,AWOX,1,740.00,740.00,0.0,0.0,740.00,NaN,Henüz fatura kesilmemiştir.
88793,78541216497852,2306187379,Trendyol Express Marketplace,9381946692,İstanbul,Pendik,"Aralıklı Plastik kedi Kumu Küreği,Temizleme El...",NaN,GOBYPET,1,31.00,31.00,2.7,0.0,28.30,NaN,Henüz fatura kesilmemiştir.
88794,604808214180,2306187379,Trendyol Express Marketplace,9381946692,İstanbul,Pendik,"Ahşap Özel Isim Baskılı, Oval Kedi Köpek Mama ...",NaN,GOBYPET,1,199.00,199.00,17.3,0.0,181.70,NaN,Henüz fatura kesilmemiştir.


In [21]:
df_kargo_other.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88796 entries, 0 to 88795
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Barkod                    88796 non-null  object 
 1   Paket No                  88796 non-null  int64  
 2   Kargo Firması             88795 non-null  object 
 3   Sipariş Numarası          88796 non-null  int64  
 4   İl                        88796 non-null  object 
 5   İlçe                      88796 non-null  object 
 6   Ürün Adı                  88796 non-null  object 
 7   Komisyon Oranı            88792 non-null  float64
 8   Marka                     88678 non-null  object 
 9   Adet                      88796 non-null  int64  
 10  Birim Fiyatı              88796 non-null  float64
 11  Satış Tutarı              88796 non-null  float64
 12  İndirim Tutarı            88796 non-null  float64
 13  Trendyol İndirim Tutarı   88796 non-null  float64
 14  Fatura

In [22]:
# df_kargo_other = df_kargo_other.drop_duplicates()
# df_kargo_other

In [23]:
def change_cargo_name(var):
    '''
    This function converts cargo names.
    '''
    if var == "Trendyol Express Marketplace":
        return 'TEX'
    elif var == "MNG Kargo Marketplace":
        return 'MNG'
    elif var == 'Yurtiçi Kargo Marketplace':
        return 'Yurtiçi'
    elif var == 'PTT Kargo Marketplace':
        return 'PTT'
    elif var == 'Sürat Kargo Marketplace':
        return 'Sürat'
    else:
        return var

In [24]:
df_kargo_other["Kargo Firması"] = df_kargo_other["Kargo Firması"].apply(lambda var: change_cargo_name(var))
df_kargo_other.head(5)

,Barkod,Paket No,Kargo Firması,Sipariş Numarası,İl,İlçe,Ürün Adı,Komisyon Oranı,Marka,Adet,Birim Fiyatı,Satış Tutarı,İndirim Tutarı,Trendyol İndirim Tutarı,Faturalanacak Tutar,Kargodan alınan desi,Faturalanan Kargo Tutarı
0,97815659247966334,1935755636,TEX,8061482665,Aydın,Germencik,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, Sakl...",20.34,ALİTECH,1,375.0,375.0,0.0,0.0,375.0,9.0,97.62
1,ALS3463102806,1935764998,TEX,2424519644,Adıyaman,Gölbaşı,3'lü Çok Amaçlı Kutu Mega Çizgili Kahverengi 6...,16.20,ALAS,1,350.0,350.0,0.0,0.0,350.0,10.0,103.37
2,604808214197,1935781088,MNG,8061494292,İstanbul,Çekmeköy,"Ahşap Özel İsim Baskılı, Kare Kedi Köpek Mama ...",18.31,GOBYPET,1,149.0,149.0,0.0,0.0,149.0,2.0,35.49
3,ÇG742305202381,1935783700,MNG,2424528751,İstanbul,Kadıköy,"Çg74 Ahşap Çocuk Makyaj Masası Tabureli,unicor...",16.27,ROZİTOYS,1,1200.0,1200.0,0.0,0.0,1200.0,10.0,112.79
4,868407600769,1935789958,MNG,8061497847,İstanbul,Ümraniye,Çömlek Ayaklı Sırlı Seramik Kedi Maması Kabı 2...,18.31,GOBYPET,1,160.0,160.0,0.0,0.0,160.0,4.0,71.99


In [25]:
cargo_price_check('Surat', 125.99)

44.99

In [26]:
df_kargo_other["Kargo Fiyat Diğer"] = df_kargo_other.apply(lambda row: cargo_price_check(row['Kargo Firması'], row['Birim Fiyatı']), axis=1)
df_kargo_other

,Barkod,Paket No,Kargo Firması,Sipariş Numarası,İl,İlçe,Ürün Adı,Komisyon Oranı,Marka,Adet,Birim Fiyatı,Satış Tutarı,İndirim Tutarı,Trendyol İndirim Tutarı,Faturalanacak Tutar,Kargodan alınan desi,Faturalanan Kargo Tutarı,Kargo Fiyat Diğer
0,97815659247966334,1935755636,TEX,8061482665,Aydın,Germencik,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, Sakl...",20.34,ALİTECH,1,375.00,375.00,0.0,0.0,375.00,9.0,97.62,NaN
1,ALS3463102806,1935764998,TEX,2424519644,Adıyaman,Gölbaşı,3'lü Çok Amaçlı Kutu Mega Çizgili Kahverengi 6...,16.20,ALAS,1,350.00,350.00,0.0,0.0,350.00,10.0,103.37,NaN
2,604808214197,1935781088,MNG,8061494292,İstanbul,Çekmeköy,"Ahşap Özel İsim Baskılı, Kare Kedi Köpek Mama ...",18.31,GOBYPET,1,149.00,149.00,0.0,0.0,149.00,2.0,35.49,44.99
3,ÇG742305202381,1935783700,MNG,2424528751,İstanbul,Kadıköy,"Çg74 Ahşap Çocuk Makyaj Masası Tabureli,unicor...",16.27,ROZİTOYS,1,1200.00,1200.00,0.0,0.0,1200.00,10.0,112.79,NaN
4,868407600769,1935789958,MNG,8061497847,İstanbul,Ümraniye,Çömlek Ayaklı Sırlı Seramik Kedi Maması Kabı 2...,18.31,GOBYPET,1,160.00,160.00,0.0,0.0,160.00,4.0,71.99,44.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88791,003548003548,2306175520,TEX,9381935071,Tekirdağ,Çerkezköy,"Yüzük Zikirmatik Karışık Renk, Yeni Nesil Ziki...",22.50,Pikas,1,29.99,29.99,0.0,0.0,29.99,NaN,Henüz fatura kesilmemiştir.,19.58
88792,8680998014047,2306177666,TEX,9381935705,Bitlis,Adilcevaz,"Rosegold Kafija Kahve Makinesi KAFİJA, one size",11.00,AWOX,1,740.00,740.00,0.0,0.0,740.00,NaN,Henüz fatura kesilmemiştir.,NaN
88793,78541216497852,2306187379,TEX,9381946692,İstanbul,Pendik,"Aralıklı Plastik kedi Kumu Küreği,Temizleme El...",NaN,GOBYPET,1,31.00,31.00,2.7,0.0,28.30,NaN,Henüz fatura kesilmemiştir.,19.58
88794,604808214180,2306187379,TEX,9381946692,İstanbul,Pendik,"Ahşap Özel Isim Baskılı, Oval Kedi Köpek Mama ...",NaN,GOBYPET,1,199.00,199.00,17.3,0.0,181.70,NaN,Henüz fatura kesilmemiştir.,37.49


In [27]:
df_kargo_other2 = df_kargo_other.dropna(subset="Kargo Fiyat Diğer")
df_kargo_other2

,Barkod,Paket No,Kargo Firması,Sipariş Numarası,İl,İlçe,Ürün Adı,Komisyon Oranı,Marka,Adet,Birim Fiyatı,Satış Tutarı,İndirim Tutarı,Trendyol İndirim Tutarı,Faturalanacak Tutar,Kargodan alınan desi,Faturalanan Kargo Tutarı,Kargo Fiyat Diğer
2,604808214197,1935781088,MNG,8061494292,İstanbul,Çekmeköy,"Ahşap Özel İsim Baskılı, Kare Kedi Köpek Mama ...",18.31,GOBYPET,1,149.00,149.00,0.0,0.0,149.00,2.0,35.49,44.99
4,868407600769,1935789958,MNG,8061497847,İstanbul,Ümraniye,Çömlek Ayaklı Sırlı Seramik Kedi Maması Kabı 2...,18.31,GOBYPET,1,160.00,160.00,0.0,0.0,160.00,4.0,71.99,44.99
5,868407600770,1935789958,MNG,8061497847,İstanbul,Ümraniye,Çömlek Ayaklı Sırlı Seramik Kedi Maması Kabı 2...,18.31,GOBYPET,1,174.00,174.00,0.0,0.0,174.00,4.0,71.99,44.99
6,ALAS0503050909,1935794248,TEX,8061500148,Gaziantep,Şahinbey,4'lü Çok Amaçlı Dolap Içi Ve Çekmece Içi Düzen...,21.36,ALAS,1,105.00,105.00,0.0,0.0,105.00,1.0,29.49,19.58
7,8-ABALAN80X30cm,1935812652,MNG,2424540787,Mardin,Artuklu,"Dağ Dekoratif Tablo, Ev Dekorasyonu, Oturma Od...",21.36,Pikas,1,165.00,165.00,0.0,0.0,165.00,2.0,58.79,44.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88788,00423564139,2306161157,TEX,9381920642,İstanbul,Bayrampaşa,"Paslanmaz Sarımsak Ezici, Sarımsak Ezeceği Met...",15.50,Pikas,1,79.00,79.00,0.0,0.0,79.00,NaN,Henüz fatura kesilmemiştir.,19.58
88789,6485371498524,2306161157,TEX,9381920642,İstanbul,Bayrampaşa,Kedi Oyun Topu Tüylü Fareli Hacı Yatmaz KT-231...,19.00,CMT,1,59.00,59.00,0.0,0.0,59.00,NaN,Henüz fatura kesilmemiştir.,19.58
88791,003548003548,2306175520,TEX,9381935071,Tekirdağ,Çerkezköy,"Yüzük Zikirmatik Karışık Renk, Yeni Nesil Ziki...",22.50,Pikas,1,29.99,29.99,0.0,0.0,29.99,NaN,Henüz fatura kesilmemiştir.,19.58
88793,78541216497852,2306187379,TEX,9381946692,İstanbul,Pendik,"Aralıklı Plastik kedi Kumu Küreği,Temizleme El...",NaN,GOBYPET,1,31.00,31.00,2.7,0.0,28.30,NaN,Henüz fatura kesilmemiştir.,19.58


In [28]:
df_kargo_other2.to_excel(r"C:\Users\user\Downloads\Trendyol Kargo Diğer Fiyat.xlsx", index=False)

##### Barkod Diff Temp 

In [74]:
df_product_master = pd.read_excel(r"C:\Users\user\Desktop\Fiyat Bilgisi Olmayan\Entegra Ürün Satış Listesi Fiyat Barkod Düzenleme Master.xlsx")
df_product_master

,Ürün Adı,Barkod Kargo,Varyant,Marka,Renk,Model Kodu,Stok Kodu,SKU,Barkod,Gelişi,KDV ORANI
0,"%100 Doğal At Kılı Selülit Fırçası, Doğal At K...",NaN,NaN,proclis,NaN,002972,002972,NaN,NaN,60,20.0
1,"%100 Doğal Keçi Kılı Yüz Bakım Fırçası, Yüz Ba...",yüz487514280114,NaN,proclis,Ahşap,002975,002975,NaN,NaN,25,20.0
2,%100 Doğal Sarı Kantaron Yağı 50 ml,8684076009987,NaN,proclis,NaN,002960,002960,NaN,NaN,28.74,20.0
3,004235,NaN,NaN,Love in Home,NaN,004235,004235,NaN,NaN,36,20.0
4,0-3 Yaş Zeka Kartları Dokun Hisset Tanı Eğitim...,2997778555322,NaN,Circle Toys,NaN,002932,002932,NaN,NaN,118,20.0
...,...,...,...,...,...,...,...,...,...,...,...
2028,Yuvarlak Torbalı Oyuncak/kirli Çamaşır Sepeti ...,0031078753,NaN,ALAS,Gri,003107,003107,NaN,NaN,92,10.0
2029,"Yuvarlak Yılan Zincir Bileklik,Charm Uyumlu",0039595743,NaN,Pikas,Gümüş,003959,003959,NaN,NaN,15,10.0
2030,Yüzük Zikirmatik Karışık Renk,003548003548,NaN,HAYATINIZI KOLAYLAŞTIRIN,Çok Renkli,003548,003548,NaN,NaN,6.5,20.0
2031,"Yüzük Zikirmatik Karışık Renk, Yeni Nesil Ziki...",003548003548,NaN,Pikas,Renkli,003548,003548,NaN,NaN,6.5,20.0


In [75]:
df_product_master.dropna(subset="Barkod Kargo", inplace=True)

In [76]:
df_product = pd.read_excel(r"C:\Users\user\Downloads\Ürünleriniz.xlsx")
df_product

c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,Partner ID,Barkod,Komisyon Oranı,Model Kodu,Ürün Rengi,Beden,Boyut/Ebat,Cinsiyet,Marka,Kategori İsmi,...,Görsel 4,Görsel 5,Görsel 6,Görsel 7,Görsel 8,Sevkiyat Süresi,Sevkiyat Tipi,Durum,Ne Yapmalıyım,Trendyol.com Linki
0,356254,CMT-BDA86FEE,19.0,TYC9bebaabb08c2db4fdb9aa,NaN,NaN,NaN,NaN,CMT,Kedi Fırça ve Tarağı,...,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,https://www.trendyol.com/cmt/kedi-kopek-bit-ve...
1,356254,65C8870EA,15.0,003878,NaN,NaN,NaN,NaN,Pikas,Nem Alıcı,...,https://cdn.dsmcdn.com/ty1457/product/media/im...,https://cdn.dsmcdn.com/ty1456/product/media/im...,NaN,NaN,NaN,2.0,NaN,NaN,NaN,https://www.trendyol.com/pikas/rutubet-gideric...
2,356254,0044754618941,19.0,004479,NaN,NaN,NaN,NaN,GOBYPET,Köpek Sağlık Ürünü,...,https://cdn.dsmcdn.com/ty1456/product/media/im...,https://cdn.dsmcdn.com/ty1456/product/media/im...,https://cdn.dsmcdn.com/ty1456/product/media/im...,NaN,NaN,2.0,NaN,NaN,NaN,https://www.trendyol.com/gobypet/kedi-kopek-bi...
3,356254,TYB8YSB1B8EQ8GKA81,11.0,TYC43A16BC9EF5EE01,Mor,NaN,NaN,NaN,AWOX,Çay Makinesi,...,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,https://www.trendyol.com/awox/teaplus-murdum-2...
4,356254,8787543541093,19.0,002837,Renkli,S,NaN,NaN,Pikas,Köpek Tasması,...,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,https://www.trendyol.com/pikas/gokkusagi-kedi-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1527,356254,8680998011633,11.0,Mercan,No Color,NaN,NaN,NaN,AWOX,Su Isıtıcı&Kettle,...,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,https://www.trendyol.com/awox/mercan-kettle-1-...
1528,356254,6941057459899,13.5,IDY59895_m,Mavi,NaN,NaN,NaN,Intex,Şişme Havuz & Aksesuarları,...,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,https://www.trendyol.com/intex/bardakli-mavi-k...
1529,356254,6941057402161,13.5,58642,(No Color),NaN,NaN,Unisex,Intex,Deniz Malzemeleri,...,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,https://www.trendyol.com/intex/intex-kirmizi-d...
1530,356254,8694064009691,16.5,2016ST023123228553951,NaN,NaN,NaN,NaN,HOBBY LİFE,Saklama Kabı,...,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,https://www.trendyol.com/hobby-life/dikdortgen...


In [77]:
df_mikro_stok = pd.read_excel(r"C:\Users\user\Desktop\Data Analysis\Adoba\Sales_Report\Data\Mikro\Stok\01.08.2024\Mikro Stok Barkod Listesi.xlsx")
df_mikro_stok

,BAĞLANTI,STOK KODU,STOK İSMİ,BARKOD,TİPİ,İÇERİK,PARTİ,LOT,BAĞ KODU,BİRİM,ASORTİ TANIM KODU,ASORTİ TANIM AÇIKLAMASI,RENK,BEDEN,MASTER
0,Stok barkodu,WAN HCN300801L,FAR SOL LEDLI MOTORLU 3008/5008,WAN HCN300801L,Ean-13,Kod,NaN,0,NaN,ADET,NaN,NaN,NaN,NaN,Yanlış
1,Stok barkodu,WAN HCN300801R,FAR SAG LEDLI MOTORLU 3008/5008,WAN HCN300801R,Ean-13,Kod,NaN,0,NaN,ADET,NaN,NaN,NaN,NaN,Yanlış
2,Stok barkodu,WAN HCN30101L,FAR SOL MOTORLU 301 2017-> MAKYAJLI KASA,WAN HCN30101L,Ean-13,Kod,NaN,0,NaN,ADET,NaN,NaN,NaN,NaN,Yanlış
3,Stok barkodu,WAN HCN30101R,FAR SAG MOTORLU 301 2017-> MAKYAJLI KASA,WANHCN30101R,Ean-13,Kod,NaN,0,NaN,ADET,NaN,NaN,NaN,NaN,Yanlış
4,Stok barkodu,WAN HCPTN01 L,FAR SOL MOTORLU PARTNER ( ledli modeller için ),WAN HCPTN01 L,Ean-13,Kod,NaN,0,NaN,ADET,NaN,NaN,NaN,NaN,Yanlış
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9788,Stok barkodu,004488,peluş halı,00448854561,Ean-13,Kod,NaN,0,NaN,ADET,NaN,NaN,NaN,NaN,Yanlış
9789,Stok barkodu,004489,kaymaz taban,00448612156,Ean-13,Kod,NaN,0,NaN,ADET,NaN,NaN,NaN,NaN,Yanlış
9790,Stok barkodu,004490,kaymaz taban 2,00449505215,Ean-13,Kod,NaN,0,NaN,ADET,NaN,NaN,NaN,NaN,Yanlış
9791,Stok barkodu,004491,kaymaz taban 3,00491625641,Ean-13,Kod,NaN,0,NaN,ADET,NaN,NaN,NaN,NaN,Yanlış


In [78]:
df_mikro_stok["BARKOD"].nunique()

9793

In [79]:
df_trend_kargo_desi_min_max_mean_most["Barkod"] = df_trend_kargo_desi_min_max_mean_most["Barkod"].apply(lambda x: convert_one_character_letter(x))
df_trend_kargo_desi_min_max_mean_most["Barkod"] = df_trend_kargo_desi_min_max_mean_most["Barkod"].apply(lambda x: whitespace_del(x))
df_trend_kargo_desi_min_max_mean_most["Barkod"] = df_trend_kargo_desi_min_max_mean_most["Barkod"].apply(lambda x: lower_func(x))

In [80]:
df_mikro_stok["BARKOD"] = df_mikro_stok["BARKOD"].apply(lambda x: convert_one_character_letter(x))
df_mikro_stok["BARKOD"] = df_mikro_stok["BARKOD"].apply(lambda x: whitespace_del(x))
df_mikro_stok["BARKOD"] = df_mikro_stok["BARKOD"].apply(lambda x: lower_func(x))

In [81]:
df_product["Barkod"] = df_product["Barkod"].apply(lambda x: convert_one_character_letter(x))
df_product["Barkod"] = df_product["Barkod"].apply(lambda x: whitespace_del(x))
df_product["Barkod"] = df_product["Barkod"].apply(lambda x: lower_func(x))

In [82]:
df_product_master["Barkod Kargo"] = df_product_master["Barkod Kargo"].apply(lambda x: convert_one_character_letter(x))
df_product_master["Barkod Kargo"] = df_product_master["Barkod Kargo"].apply(lambda x: whitespace_del(x))
df_product_master["Barkod Kargo"] = df_product_master["Barkod Kargo"].apply(lambda x: lower_func(x))

In [83]:
trend_kargo_set = set(df_trend_kargo_desi_min_max_mean_most["Barkod"])
micro_stok_set = set(df_mikro_stok["BARKOD"])
product_set = set(df_product["Barkod"])
product_master_set = set(df_product_master["Barkod Kargo"])

In [84]:
trend_kargo_set.difference(micro_stok_set)

{'0025521042103',
 '0038374461',
 '02032023089',
 '02050620235',
 '02062023004',
 '02062023055',
 '0206202323048',
 '050506202314',
 '050605202314',
 '05062023007',
 '0506202306',
 '0604808214739',
 '0789431451529',
 '1111111111111',
 '1204202311128',
 '1234555574214',
 '1234567a038',
 '199080001533',
 '239013497520166',
 '2401201608021',
 '2592412160240-1',
 '3-8699243469945-ly3',
 '311764500047',
 '5352004838581',
 '57568678',
 '6048082218719',
 '6485371498524',
 '6941057412436',
 '6941057412438',
 '6941057454412',
 '71300000800',
 '713002932',
 '7658456443265375',
 '7805062026',
 '8-abalan80x30cm',
 '8417781982174',
 '85369541824630',
 '85369541824634',
 '85646541685',
 '8680067147850',
 '8680998011022',
 '8680998011091',
 '868099801117855',
 '8680998012128',
 '8680998014047',
 '8680998014122',
 '8680998014153',
 '8680998014467',
 '8680998014502',
 '8680998014542',
 '8680998018441',
 '8680998018502',
 '8680998018540',
 '8680998224023',
 '86814530000033',
 '8681475501213',
 '86819425

In [85]:
trend_kargo_set.difference(product_set)

{'00262122110706',
 '002643limon',
 '003080gri218545',
 '003081bej',
 '003082bej820030',
 '003100852541',
 '003165lila',
 '0032199354',
 '0032866764524',
 '0033061845',
 '0033534897854',
 '00336241745',
 '00336267894',
 '00341894644531',
 '003430034643',
 '00346574134',
 '0034667916',
 '0034704416441',
 '0034794589',
 '0034851367943',
 '003494865123587',
 '00349684617631756',
 '00351187954',
 '0035279846231',
 '0035527579',
 '003553785463',
 '003554872641',
 '003557894891',
 '0035587941654',
 '003562479456',
 '00356347418',
 '0035700248',
 '0035714186745',
 '003598874541145',
 '0036287984651',
 '003629784144589',
 '00363141676',
 '00363241871',
 '00363341823',
 '003636744894',
 '0036634681231',
 '003666187321',
 '00367046545842458',
 '00376578945',
 '0037669416',
 '003767048152',
 '003771458456',
 '0038248641142',
 '0038501613',
 '003856412',
 '0038657545',
 '003880146763',
 '00388341643',
 '003885534313',
 '00388741',
 '00388943132',
 '00394434135',
 '0039547513',
 '003958474231343',


In [86]:
product_master_set.difference(micro_stok_set)

{'0025521042103',
 '0038374461',
 '02032023089',
 '02050620235',
 '02062023003',
 '02062023004',
 '02062023055',
 '0206202323048',
 '050605202314',
 '05062023007',
 '0506202306',
 '0604808214739',
 '0789431451529',
 '1111111111111',
 '1204202311128',
 '1234555574214',
 '1234567a038',
 '19900800t0631',
 '239013497520166',
 '2401201608021',
 '2592412160240-1',
 '3-8699243469945-ly3',
 '5352004838581',
 '57568678',
 '6048082218719',
 '6941057412436',
 '6941057412438',
 '6941057454412',
 '6968564315263168',
 '71300000800',
 '713002932',
 '8-abalan80x30cm',
 '85369541824630',
 '85646541685',
 '8680067147850',
 '8680998011022',
 '868099801117855',
 '8680998014047',
 '8680998014153',
 '8680998014502',
 '8680998014542',
 '8680998018441',
 '8680998018540',
 '8680998224023',
 '8681475501213',
 '8681942503963-1',
 '8682315102319',
 '8682315110456-1',
 '8682315115048',
 '8682315115062',
 '8682511805972',
 '8683873040001',
 '8684076000298-2',
 '8694359083917-1',
 '8695614350874',
 '8695614351437',


In [87]:
product_master_mikro_diff = list(product_master_set.difference(micro_stok_set))

In [88]:
trend_kargo_mikro_diff = list(trend_kargo_set.difference(micro_stok_set))

In [89]:
df_trend_kargo_mikro_diff = df_trend_kargo_desi_min_max_mean_most[df_trend_kargo_desi_min_max_mean_most["Barkod"].isin(trend_kargo_mikro_diff)]
df_trend_kargo_mikro_diff

,Barkod,Paket No,Kargo Firması,Sipariş Numarası,İl,İlçe,Ürün Adı,Komisyon Oranı,Marka,Adet,...,Faturalanan Kargo Tutarı,kargo desi min,kargo desi max,kargo desi ortalama,kargo desi en sık,count,Kargo Fiyat Min,Kargo Fiyat Max,Kargo Fiyat Ortalama,Kargo Fiyat En Sık
16,0025521042103,2009337646,TEX,9046774217,Şanlıurfa,Karaköprü,En Sevilen Aromaterapi Sabit Yağlar Paketi ( A...,16.27,proclis,1,...,44.99,1.0,1.0,1.0,1.0,1,53.340,53.340,53.340,53.340
259,0038374461,2238211827,TEX,9304788471,Eskişehir,Tepebaşı,Kıvrımlı Kedi Tüneli Katlanır Gökkuşağı S Kıvr...,18.31,GOBYPET,1,...,53.34,1.0,2.0,2.0,1.0,2,53.340,53.340,53.340,53.340
260,0038374461,2225773652,TEX,9289896957,İstanbul,Sancaktepe,Kıvrımlı Kedi Tüneli Katlanır Gökkuşağı S Kıvr...,18.31,AKIN PAZAR,1,...,53.34,1.0,3.0,2.0,3.0,1,53.340,62.640,53.340,62.640
433,02032023089,2024198145,TEX,9064210372,İstanbul,Esenyurt,En Sevilen Aromaterapi Uçucu Yağları Paketi ( ...,16.27,proclis,1,...,53.34,1.0,1.0,1.0,1.0,2,53.340,53.340,53.340,53.340
434,02050620235,1947963259,MNG,2432390327,Batman,Merkez,En Sevilen Aromaterapi Uçucu Yağları Paketi( N...,16.27,proclis,1,...,65.99,3.0,3.0,3.0,3.0,1,68.388,68.388,68.388,68.388
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2466,çakmakgazı11042023-4,2045832270,PTT,9090581244,Muğla,Ortaca,Çakmak Gazı 270 Ml. Parlı Kırmızı Kutu 3'lü Se...,16.27,Flash,1,...,23.49,2.0,4.0,3.0,2.0,8,57.264,70.896,70.896,57.264
2467,çakmakgazı11042023-4,2042258625,MNG,9086468292,Çanakkale,Merkez,Çakmak Gazı 270 Ml. Parlı Kırmızı Kutu 3'lü Se...,16.27,Flash,1,...,30.99,2.0,4.0,3.0,2.0,8,62.388,75.588,68.388,62.388
2468,çakmakgazı11042023-6,2291008030,Yurtiçi,9364556968,İstanbul,Beylikdüzü,Çakmak Gazı 270 Ml. Parlı Kırmızı Kutu 5'li Se...,16.50,Flash,1,...,Henüz fatura kesilmemiştir.,1.0,4.0,3.0,2.0,4,75.972,82.968,81.216,75.972
2469,çakmakgazı11042023-6,2052439303,PTT,9098432861,Niğde,Merkez,Çakmak Gazı 270 Ml. Parlı Kırmızı Kutu 5'li Se...,16.27,Flash,1,...,44.99,1.0,4.0,3.0,2.0,4,57.264,70.896,70.896,57.264


In [90]:
# df_trend_kargo_mikro_diff.to_excel(r"C:\Users\user\Downloads\Trend Kargo Barkod Mikro Stok Barkodda Olmayanlar.xlsx", index=False)

In [91]:
df_product_master_mikro_diff = df_product_master[df_product_master["Barkod Kargo"].isin(product_master_mikro_diff)]
df_product_master_mikro_diff

,Ürün Adı,Barkod Kargo,Varyant,Marka,Renk,Model Kodu,Stok Kodu,SKU,Barkod,Gelişi,KDV ORANI
7,1 Adet Kapaklı Kutu Hurç Mega Boy 60*40*30*,713002932,NaN,ALAS,Gri,001605,001605,NaN,NaN,64,10.0
19,"11 Parça Emaye Tencere Tava Set Pembe, Oyuncak...",8694359083917-1,NaN,ROZİTOYS,NaN,004074,004074,NaN,NaN,326,20.0
21,114 Cm Kristal Havuz + 100 Adet 6cm Top + Pompa,tybp60ui2b5mzf9h42,NaN,Intex,Mavi,TYBP60UI2B5MZF9H42,TYBP60UI2B5MZF9H42,NaN,NaN,247.5,20.0
25,2 Adet - Kapaklı Çok Amaçlı Hurç Kutu Mega Plu...,keven876422,NaN,ALAS,Kahverengi,002943,002943,NaN,NaN,176,10.0
26,2 Adet Baza Içi Düzenleyici Kapaklı Organizer ...,hurç504019,NaN,ALAS,KAHVERENGİ,002941,002941,NaN,NaN,122,10.0
...,...,...,...,...,...,...,...,...,...,...,...
1900,T2013 Ahşap Kaydıraklı Oyun Evi,8682315115062,NaN,Toysilla,Çok Renkli,004206,004206,NaN,NaN,800,20.0
1905,Tamir Onarım Yaması 3 Adet ( 18'li ) Şişme Bot...,8699329799743,NaN,Intex,Beyaz,004198,004198,NaN,NaN,16.5,20.0
1920,Tek Kişilik Şişme Yatak Lacivert,6941057412436,NaN,Intex,"Karışık, Çok Renkli",004001,004001,NaN,NaN,290,20.0
1952,Turkuaz Kafija Kahve Makinesi New Sesli Ve Işı...,8680998014153,NaN,AWOX,TURKUAZ,003565,003565,NaN,NaN,344,20.0


In [92]:
# df_product_master_mikro_diff.to_excel(r"C:\Users\user\Downloads\Product Master Barkod Mikro Stok Barkodda Olmayanlar.xlsx", index=False)

##### Temp

In [93]:
df_trend_kargo2.head()

,Barkod,Paket No,Kargo Firması,Sipariş Numarası,İl,İlçe,Ürün Adı,Komisyon Oranı,Marka,Adet,...,İndirim Tutarı,Trendyol İndirim Tutarı,Faturalanacak Tutar,Kargodan alınan desi,Faturalanan Kargo Tutarı,kargo desi min,kargo desi max,kargo desi ortalama,kargo desi en sık,count
0,000416648254,2280482994,TEX,9352539467,İzmir,Gaziemir,"Pembe Ahşap Kaydıraklı Oyun Evi, Ahşap Oyun Ev...",13.50,ROZİTOYS,1,...,10.0,0,1890.00,9.0,Henüz fatura kesilmemiştir.,9.0,27.0,20.0,27.0,2
1,000437943412,2219946411,TEX,9283284388,İstanbul,Bakırköy,Çamaşır Kurutma Makinası İçin Uçucu Yağlar Set...,12.20,proclis,1,...,0.0,0,270.00,1.0,53.34,1.0,2.0,2.0,2.0,3
2,0006941057446691,2082678312,TEX,9130023184,Ankara,Yenimahalle,Hayvanlı Kumsuz Hacıyatmaz Kaplan 98 Cm X 44 C...,13.73,Intex,1,...,0.0,0,285.00,1.0,53.34,1.0,2.0,2.0,1.0,2
3,0006941057446691,2176825666,MNG,9234601826,Eskişehir,Alpu,Hayvanlı Kumsuz Hacıyatmaz Kaplan 98 Cm X 44 C...,13.73,Intex,1,...,0.0,0,285.00,2.0,58.79,1.0,2.0,2.0,1.0,2
4,002154600346,2116926027,TEX,9167602070,İstanbul,Başakşehir,"Dekoratif Salyangoz Jüt Dokuma Bardak Altlığı,...",19.32,Pikas,1,...,0.0,0,249.99,8.0,94.06,8.0,8.0,8.0,8.0,1


In [94]:
df_kargo_desi_fiyat

,Desi/KG,Aras,MNG,PTT,Sendeo,Sürat,TEX,Yurtiçi,Borusan,CEVA,Horoz
0,0,56.988,62.388,57.264,59.160,58.464,53.340,75.972,392.412,496.524,464.256
1,1,56.988,62.388,57.264,59.160,58.464,53.340,75.972,392.412,496.524,464.256
2,2,61.284,62.388,57.264,59.160,58.464,53.340,75.972,392.412,496.524,464.256
3,3,68.652,68.388,70.896,70.452,67.692,62.640,81.216,392.412,496.524,464.256
4,4,74.832,75.588,70.896,72.852,74.400,67.128,82.968,392.412,496.524,464.256
...,...,...,...,...,...,...,...,...,...,...,...
96,96,729.864,1535.196,1264.380,661.044,842.112,691.956,909.120,1234.176,1115.028,1019.484
97,97,737.208,1553.184,1276.248,666.516,850.584,698.796,918.132,1247.028,1126.704,1030.104
98,98,744.552,1571.172,1288.128,672.000,859.080,705.648,927.144,1259.880,1138.260,1040.724
99,99,751.896,1589.160,1299.996,677.472,867.540,712.488,936.156,1272.744,1149.936,1051.332


In [95]:
df_kargo_desi_fiyat[df_kargo_desi_fiyat["Desi/KG"] == 2][f"Sendeo"].values[0]

59.16

In [96]:
df_trend_kargo2["kargo desi min"]

0       9.0
1       1.0
2       1.0
3       1.0
4       8.0
       ... 
2466    2.0
2467    2.0
2468    1.0
2469    1.0
2470    1.0
Name: kargo desi min, Length: 2471, dtype: float64

In [97]:
df_trend_kargo2.loc[:,"Kargo Fiyat Min"] = df_trend_kargo2.apply(lambda row: df_kargo_desi_fiyat[df_kargo_desi_fiyat["Desi/KG"] == row["kargo desi min"]][f"{row["Kargo Firması"]}"].values[0], axis=1)
df_trend_kargo2.loc[:,"Kargo Fiyat Max"] = df_trend_kargo2.apply(lambda row: df_kargo_desi_fiyat[df_kargo_desi_fiyat["Desi/KG"] == row["kargo desi max"]][f"{row["Kargo Firması"]}"].values[0], axis=1)
df_trend_kargo2.loc[:,"Kargo Fiyat Ortalama"] = df_trend_kargo2.apply(lambda row: df_kargo_desi_fiyat[df_kargo_desi_fiyat["Desi/KG"] == row["kargo desi ortalama"]][f"{row["Kargo Firması"]}"].values[0], axis=1)
df_trend_kargo2.loc[:,"Kargo Fiyat En Sık"] = df_trend_kargo2.apply(lambda row: df_kargo_desi_fiyat[df_kargo_desi_fiyat.loc[:,"Desi/KG"] == row["kargo desi en sık"]][f"{row["Kargo Firması"]}"].values[0], axis=1)
df_trend_kargo2

,Barkod,Paket No,Kargo Firması,Sipariş Numarası,İl,İlçe,Ürün Adı,Komisyon Oranı,Marka,Adet,...,Faturalanan Kargo Tutarı,kargo desi min,kargo desi max,kargo desi ortalama,kargo desi en sık,count,Kargo Fiyat Min,Kargo Fiyat Max,Kargo Fiyat Ortalama,Kargo Fiyat En Sık
0,000416648254,2280482994,TEX,9352539467,İzmir,Gaziemir,"Pembe Ahşap Kaydıraklı Oyun Evi, Ahşap Oyun Ev...",13.50,ROZİTOYS,1,...,Henüz fatura kesilmemiştir.,9.0,27.0,20.0,27.0,2,97.620,206.580,160.488,206.580
1,000437943412,2219946411,TEX,9283284388,İstanbul,Bakırköy,Çamaşır Kurutma Makinası İçin Uçucu Yağlar Set...,12.20,proclis,1,...,53.34,1.0,2.0,2.0,2.0,3,53.340,53.340,53.340,53.340
2,0006941057446691,2082678312,TEX,9130023184,Ankara,Yenimahalle,Hayvanlı Kumsuz Hacıyatmaz Kaplan 98 Cm X 44 C...,13.73,Intex,1,...,53.34,1.0,2.0,2.0,1.0,2,53.340,53.340,53.340,53.340
3,0006941057446691,2176825666,MNG,9234601826,Eskişehir,Alpu,Hayvanlı Kumsuz Hacıyatmaz Kaplan 98 Cm X 44 C...,13.73,Intex,1,...,58.79,1.0,2.0,2.0,1.0,2,62.388,62.388,62.388,62.388
4,002154600346,2116926027,TEX,9167602070,İstanbul,Başakşehir,"Dekoratif Salyangoz Jüt Dokuma Bardak Altlığı,...",19.32,Pikas,1,...,94.06,8.0,8.0,8.0,8.0,1,94.068,94.068,94.068,94.068
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2466,ÇakmakGazı11042023-4,2045832270,PTT,9090581244,Muğla,Ortaca,Çakmak Gazı 270 Ml. Parlı Kırmızı Kutu 3'lü Se...,16.27,Flash,1,...,23.49,2.0,4.0,3.0,2.0,8,57.264,70.896,70.896,57.264
2467,ÇakmakGazı11042023-4,2042258625,MNG,9086468292,Çanakkale,Merkez,Çakmak Gazı 270 Ml. Parlı Kırmızı Kutu 3'lü Se...,16.27,Flash,1,...,30.99,2.0,4.0,3.0,2.0,8,62.388,75.588,68.388,62.388
2468,ÇakmakGazı11042023-6,2291008030,Yurtiçi,9364556968,İstanbul,Beylikdüzü,Çakmak Gazı 270 Ml. Parlı Kırmızı Kutu 5'li Se...,16.50,Flash,1,...,Henüz fatura kesilmemiştir.,1.0,4.0,3.0,2.0,4,75.972,82.968,81.216,75.972
2469,ÇakmakGazı11042023-6,2052439303,PTT,9098432861,Niğde,Merkez,Çakmak Gazı 270 Ml. Parlı Kırmızı Kutu 5'li Se...,16.27,Flash,1,...,44.99,1.0,4.0,3.0,2.0,4,57.264,70.896,70.896,57.264
